In [1]:
import requests
import time
from typing import Optional, Tuple

DOMAIN_JP = 5
MAX_TOKENS_ALLOWED = 10    # 既存のトークン閾値（併用する場合に有効）
MAX_SECONDS_ALLOWED = 10   # 許容する最大経過時間（秒）

def fetch_top_display_price(api_key: str, code: str, max_retries=3,
                            max_seconds: int = MAX_SECONDS_ALLOWED,
                            max_tokens: Optional[int] = MAX_TOKENS_ALLOWED
                           ) -> Tuple[Optional[str], Optional[int], int, int, int]:
    """
    ページトップに表示される価格を取得。呼び出し全体の最大許容秒数で中断可能。
    戻り値: (title, price, hit_count, tokens_consumed, tokens_left)
    """
    url = (
        # "https://api.keepa.com/product"
        # f"?key={api_key}"
        # f"&domain={DOMAIN_JP}"
        # f"&code={code}"
        # "&history=0"
        # "&offers=20"
        # "&onlyLiveOffers=0"
        # "&stats=0"


        "https://api.keepa.com/product"
        f"?key={api_key}"
        f"&domain={DOMAIN_JP}"
        f"&code={code}"
        "&history=0"
        "&offers=20"          # 出品情報は20件まで（維持）
        "&onlyLiveOffers=0"   # ✅ 再帰問い合わせ抑制
        "&buybox=1"           # ✅ BuyBox情報を直接取得
        "&stats=0"    
    )

    start = time.time()
    data = None
    for attempt in range(max_retries):
        # 経過時間チェック：リトライ前に時間切れなら中断
        elapsed = time.time() - start
        if max_seconds is not None and elapsed >= max_seconds:
            print(f"❌ 時間上限に到達しました: {elapsed:.2f}s（上限: {max_seconds}s）")
            tokens_consumed = data.get("tokensConsumed", 0) if data else 0
            tokens_left = data.get("tokensLeft", 0) if data else 0
            return None, None, 0, tokens_consumed, tokens_left

        try:
            resp = requests.get(url, timeout=30)
            resp.raise_for_status()
            data = resp.json()

            # 取得直後に経過時間チェック：レスポンス解析前に時間切れを確認
            elapsed = time.time() - start
            if max_seconds is not None and elapsed >= max_seconds:
                print(f"❌ 時間上限に到達しました（レスポンス取得後）: {elapsed:.2f}s")
                tokens_consumed = data.get("tokensConsumed", 0)
                tokens_left = data.get("tokensLeft", 0)
                return None, None, len(data.get("products", [])), tokens_consumed, tokens_left

            break
        except requests.exceptions.ReadTimeout:
            print(f"⚠️ タイムアウト（{attempt+1}/{max_retries}）: {code}")
            time.sleep(1)
        except Exception as e:
            print(f"⚠️ エラー: {e}")
            return None, None, 0, 0, 0

    if not data or "products" not in data or not data["products"]:
        print("❌ 商品情報が見つかりませんでした")
        tokens_consumed = data.get("tokensConsumed", 0) if data else 0
        tokens_left = data.get("tokensLeft", 0) if data else 0
        return None, None, 0, tokens_consumed, tokens_left

    tokens_consumed = data.get("tokensConsumed", 0)
    tokens_left = data.get("tokensLeft", 0)

    # オプションでトークン閾値も併用する
    if max_tokens is not None and tokens_consumed >= max_tokens:
        print(f"❌ トークン消費が多すぎます: {tokens_consumed}（許容上限: {max_tokens - 1}）")
        return None, None, len(data.get("products", [])), tokens_consumed, tokens_left

    hit_count = len(data["products"])
    product = data["products"][0]
    title = product.get("title", "タイトルなし")

    stats = product.get("stats") or {}
    for key in ("buyBoxShippingPrice", "buyBoxPrice", "current_BUY_BOX_SHIPPING"):
        v = stats.get(key)
        if isinstance(v, (int, float)) and v > 0:
            return title, int(v), hit_count, tokens_consumed, tokens_left

    offers = product.get("offers") or []
    order = product.get("liveOffersOrder") or []
    ordered = [offers[i] for i in order if isinstance(i, int) and i < len(offers)]
    if not ordered and offers:
        ordered = offers

    prime_offer = next((o for o in ordered if o.get("isPrime")), None)
    chosen = prime_offer or (ordered[0] if ordered else None)

    if chosen:
        p = chosen.get("price")
        ship = chosen.get("shipping")
        if isinstance(p, (int, float)) and p > 0:
            total = int(p) + (int(ship) if isinstance(ship, (int, float)) and ship > 0 else 0)
            return title, total, hit_count, tokens_consumed, tokens_left

    return None, None, hit_count, tokens_consumed, tokens_left


# 実行例は元のまま
if __name__ == "__main__":
    API_KEY = "5evt1mqp5d7ju3q7kmlt8s27lp0gf8n51oird2ivf6b8oj1ko5s2ltnd2n9dgo9j"
    JAN = "4522654180816"
    title, price, hit_count, tokens_used, tokens_left = fetch_top_display_price(API_KEY, JAN)
    print("────────────────────────────")
    print(f"ヒット件数: {hit_count} 件")
    print("商品コード" + JAN)
    if title and price:
        print(f"商品タイトル: {title}")
        print(f"ページトップ表示価格: {price} 円")
    else:
        print("価格取得失敗")
    print("────────────────────────────")
    print(f"消費トークン: {tokens_used}")
    print(f"残トークン数: {tokens_left}")
    print("────────────────────────────")


────────────────────────────
ヒット件数: 1 件
商品コード4522654180816
商品タイトル: 森本産業 ﾋﾞｼﾞｭｰﾊﾞﾝｽｸﾘｯﾌﾟ ｻﾝﾘｵ ｸﾛﾐ RM-8436 H7.5×W8.2×D5.4cm
ページトップ表示価格: 1620 円
────────────────────────────
消費トークン: 6
残トークン数: 294
────────────────────────────


In [6]:
import requests
import time
import pandas as pd
import tkinter as tk
from tkinter import messagebox
from tkinterdnd2 import DND_FILES, TkinterDnD
from typing import Optional, Tuple
import threading

# ----------------------------------------
# 基本設定
# ----------------------------------------
DOMAIN_JP = 5
MAX_SECONDS_ALLOWED = 10
TOKEN_WAIT_TIME = 1800  # トークン切れ時の待機秒数（30分）

# ----------------------------------------
# Keepa API 取得関数
# ----------------------------------------
def fetch_top_display_price(api_key: str, code: str) -> Tuple[Optional[str], Optional[int], Optional[str], int, int]:
    url = (
        f"https://api.keepa.com/product?key={api_key}"
        f"&domain={DOMAIN_JP}&code={code}"
        "&history=0&offers=20&onlyLiveOffers=0&buybox=1&stats=0"
    )

    try:
        resp = requests.get(url, timeout=20)
        data = resp.json()
    except Exception as e:
        return None, None, f"通信エラー: {e}", 0, 0

    if not data or "products" not in data:
        return None, None, "データなし", 0, 0

    tokens_consumed = data.get("tokensConsumed", 0)
    tokens_left = data.get("tokensLeft", 0)
    products = data["products"]

    if not products:
        return None, None, "商品情報なし", tokens_consumed, tokens_left

    product = products[0]
    title = product.get("title", "")
    stats = product.get("stats") or {}

    # --- BuyBox価格を優先 ---
    for key in ("buyBoxPrice", "buyBoxShippingPrice", "current_BUY_BOX_SHIPPING"):
        v = stats.get(key)
        if isinstance(v, (int, float)) and v > 0:
            return title, int(v), None, tokens_consumed, tokens_left

    # --- Prime優先ロジック ---
    offers = product.get("offers") or []
    order = product.get("liveOffersOrder") or []
    ordered = [offers[i] for i in order if isinstance(i, int) and i < len(offers)]
    if not ordered and offers:
        ordered = offers

    prime_offer = next((o for o in ordered if o.get("isPrime")), None)
    chosen = prime_offer or (ordered[0] if ordered else None)

    if chosen:
        price = chosen.get("price")
        ship = chosen.get("shipping") or 0
        if price and price > 0:
            total = int(price) + int(ship)
            return title, total, None, tokens_consumed, tokens_left

    return title, None, "価格情報なし", tokens_consumed, tokens_left


# ----------------------------------------
# Excel処理メインロジック（列名なし対応）
# ----------------------------------------
def start_process(api_key, filepath, log_box):
    try:
        df = pd.read_excel(filepath, header=None)
    except Exception as e:
        messagebox.showerror("読込エラー", f"Excelファイルを開けませんでした。\n{e}")
        return

    # 列名なしのため自動命名（想定：1列目=JAN / 2列目=商品名 / 3列目=価格 / 4列目=備考）
    cols = ["JAN", "商品名", "価格", "備考"]
    while len(df.columns) < 4:
        df[len(df.columns)] = ""
    df.columns = cols[:len(df.columns)]

    total = len(df)
    log_box.insert(tk.END, f"📘 ファイル読込完了: {filepath}\n🔢 全{total}件の処理を開始します。\n\n")
    log_box.update()

    for i, row in df.iterrows():
        jan = str(row["JAN"])
        old_price = row.get("価格", None)
        if pd.notna(old_price) and old_price != "":
            log_box.insert(tk.END, f"✅ {i+1}/{total} {jan} → 価格済み（スキップ）\n")
            log_box.update()
            continue

        title, price, error, tokens_used, tokens_left = fetch_top_display_price(api_key, jan)

        # --- トークン切れ対策 ---
        if tokens_left <= 0:
            log_box.insert(tk.END, f"⏸ トークン枯渇。{TOKEN_WAIT_TIME//60}分待機中...\n")
            log_box.update()
            time.sleep(TOKEN_WAIT_TIME)

        # --- 結果をExcelに書き込み ---
        df.at[i, "商品名"] = title or row.get("商品名", "")
        df.at[i, "価格"] = price if price else row.get("価格", "")
        df.at[i, "備考"] = error or ""

        remaining = total - (i + 1)
        log_box.insert(tk.END, f"🕐 {i+1}/{total} 完了 → 残り{remaining}件 | トークン残: {tokens_left}\n")
        log_box.update()

        time.sleep(1.0)  # API連打防止

    # --- 結果保存 ---
    output_file = "結果.xlsx"
    df.to_excel(output_file, index=False, header=["JAN", "商品名", "価格", "備考"])
    log_box.insert(tk.END, f"\n🎉 完了！結果を「{output_file}」に保存しました。\n")
    messagebox.showinfo("完了", f"処理が完了しました！\n結果ファイル: {output_file}")


# ----------------------------------------
# GUI構築
# ----------------------------------------
def create_gui():
    root = TkinterDnD.Tk()
    root.title("Keepa価格取得ツール（列名なし対応）")
    root.geometry("700x500")

    tk.Label(root, text="Keepa APIキー：", font=("Meiryo", 11, "bold")).pack(anchor="w", padx=10, pady=5)
    api_entry = tk.Entry(root, width=80, show="*")
    api_entry.pack(padx=10)

    tk.Label(root, text="Excelファイルをドロップ：", font=("Meiryo", 11, "bold")).pack(anchor="w", padx=10, pady=5)
    file_label = tk.Label(root, text="（ここにドラッグ＆ドロップ）", bg="#f8f8f8",
                          width=80, height=2, relief="groove", font=("Meiryo", 10))
    file_label.pack(padx=10, pady=5)

    def drop_file(event):
        filepath = event.data.strip("{}")
        file_label.config(text=filepath)
        file_label.filepath = filepath

    file_label.drop_target_register(DND_FILES)
    file_label.dnd_bind('<<Drop>>', drop_file)

    log_box = tk.Text(root, height=18, width=90, font=("Meiryo", 9))
    log_box.pack(padx=10, pady=10)

    def on_start():
        api_key = api_entry.get().strip()
        filepath = getattr(file_label, "filepath", None)
        if not api_key or not filepath:
            messagebox.showerror("エラー", "APIキーとExcelファイルを入力してください。")
            return
        threading.Thread(target=start_process, args=(api_key, filepath, log_box), daemon=True).start()

    tk.Button(root, text="▶ 開始", command=on_start, bg="#4CAF50", fg="white",
              font=("Meiryo", 11, "bold"), width=15, height=1).pack(pady=10)

    root.mainloop()


# ----------------------------------------
# 実行エントリーポイント
# ----------------------------------------
if __name__ == "__main__":
    create_gui()


C:\Users\taku5\AppData\Local\Temp\ipykernel_35460\713198140.py:112: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '商品情報なし' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[i, "備考"] = error or ""


In [7]:
import requests
import time
import pandas as pd
import tkinter as tk
from tkinter import messagebox
from tkinterdnd2 import DND_FILES, TkinterDnD
from typing import Optional, Tuple
import threading
import datetime
import os

# ----------------------------------------
# 基本設定
# ----------------------------------------
DOMAIN_JP = 5
MAX_SECONDS_ALLOWED = 10
TOKEN_WAIT_TIME = 300       # 5分待機
AUTOSAVE_INTERVAL = 180     # 3分ごと自動保存

# ----------------------------------------
# Keepa API 取得関数
# ----------------------------------------
def fetch_top_display_price(api_key: str, code: str) -> Tuple[Optional[str], Optional[int], Optional[str], int, int]:
    url = (
        f"https://api.keepa.com/product?key={api_key}"
        f"&domain={DOMAIN_JP}&code={code}"
        "&history=0&offers=20&onlyLiveOffers=0&buybox=1&stats=0"
    )

    try:
        resp = requests.get(url, timeout=20)
        data = resp.json()
    except Exception as e:
        return None, None, f"通信エラー: {e}", 0, 0

    if not data or "products" not in data:
        return None, None, "データなし", 0, 0

    tokens_consumed = data.get("tokensConsumed", 0)
    tokens_left = data.get("tokensLeft", 0)
    products = data["products"]

    if not products:
        return None, None, "商品情報なし", tokens_consumed, tokens_left

    product = products[0]
    title = product.get("title", "")
    stats = product.get("stats") or {}

    # --- BuyBox価格を優先 ---
    for key in ("buyBoxPrice", "buyBoxShippingPrice", "current_BUY_BOX_SHIPPING"):
        v = stats.get(key)
        if isinstance(v, (int, float)) and v > 0:
            return title, int(v), None, tokens_consumed, tokens_left

    # --- Prime優先ロジック ---
    offers = product.get("offers") or []
    order = product.get("liveOffersOrder") or []
    ordered = [offers[i] for i in order if isinstance(i, int) and i < len(offers)]
    if not ordered and offers:
        ordered = offers

    prime_offer = next((o for o in ordered if o.get("isPrime")), None)
    chosen = prime_offer or (ordered[0] if ordered else None)

    if chosen:
        price = chosen.get("price")
        ship = chosen.get("shipping") or 0
        if price and price > 0:
            total = int(price) + int(ship)
            return title, total, None, tokens_consumed, tokens_left

    return title, None, "価格情報なし", tokens_consumed, tokens_left


# ----------------------------------------
# Excel処理メインロジック（自動保存対応）
# ----------------------------------------
def start_process(api_key, filepath, log_box):
    try:
        df = pd.read_excel(filepath, header=None)
    except Exception as e:
        messagebox.showerror("読込エラー", f"Excelファイルを開けませんでした。\n{e}")
        return

    # 列名なし対応：自動設定
    cols = ["JAN", "商品名", "価格", "備考"]
    while len(df.columns) < 4:
        df[len(df.columns)] = ""
    df.columns = cols[:len(df.columns)]

    total = len(df)
    start_time = time.time()
    save_timer = start_time

    # 出力ファイル名（上書きではなく新規作成）
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    output_file = f"結果_{timestamp}.xlsx"

    log_box.insert(tk.END, f"📘 ファイル読込完了: {filepath}\n")
    log_box.insert(tk.END, f"🔢 全{total}件の処理を開始します。\n\n")
    log_box.update()

    for i, row in df.iterrows():
        jan = str(row["JAN"]).strip()

        # --- 価格が既に入っている場合はスキップ ---
        old_price = row.get("価格", None)
        if pd.notna(old_price) and str(old_price).strip() != "":
            log_box.insert(tk.END, f"✅ {i+1}/{total} {jan} → 価格済み（スキップ）\n")
            log_box.update()
            continue

        # --- API呼び出し ---
        title, price, error, tokens_used, tokens_left = fetch_top_display_price(api_key, jan)

        # --- トークン枯渇対策 ---
        if tokens_left <= 0:
            msg = f"⚠️ トークン枯渇 detected！{TOKEN_WAIT_TIME//60}分待機します...\n"
            log_box.insert(tk.END, msg)
            log_box.update()
            df.at[i, "備考"] = "トークン待機中（5分）"
            df.to_excel(output_file, index=False)
            time.sleep(TOKEN_WAIT_TIME)
            log_box.insert(tk.END, "⏳ 再開しました。\n")

        # --- 結果反映 ---
        df.at[i, "商品名"] = title or row.get("商品名", "")
        df.at[i, "価格"] = price if price else row.get("価格", "")
        df.at[i, "備考"] = error or ""

        # --- 自動保存（3分ごと） ---
        now = time.time()
        if now - save_timer >= AUTOSAVE_INTERVAL:
            df.to_excel(output_file, index=False)
            log_box.insert(tk.END, "💾 自動保存しました（3分経過）\n")
            log_box.update()
            save_timer = now

        # --- 進捗ログ ---
        remaining = total - (i + 1)
        log_box.insert(tk.END, f"🕐 {i+1}/{total} 完了 → 残り{remaining}件 | トークン残: {tokens_left}\n")
        log_box.update()

        time.sleep(1.0)

    # --- 最終保存 ---
    df.to_excel(output_file, index=False)
    log_box.insert(tk.END, f"\n🎉 完了！結果を「{output_file}」に保存しました。\n")
    messagebox.showinfo("完了", f"処理が完了しました！\n結果ファイル: {output_file}")


# ----------------------------------------
# GUI構築
# ----------------------------------------
def create_gui():
    root = TkinterDnD.Tk()
    root.title("Keepa価格取得ツール（フラワーさん専用）")
    root.geometry("700x520")

    tk.Label(root, text="Keepa APIキー：", font=("Meiryo", 11, "bold")).pack(anchor="w", padx=10, pady=5)
    api_entry = tk.Entry(root, width=80, show="*")
    api_entry.pack(padx=10)

    tk.Label(root, text="Excelファイルをドロップ：", font=("Meiryo", 11, "bold")).pack(anchor="w", padx=10, pady=5)
    file_label = tk.Label(root, text="（ここにドラッグ＆ドロップ）", bg="#f8f8f8",
                          width=80, height=2, relief="groove", font=("Meiryo", 10))
    file_label.pack(padx=10, pady=5)

    def drop_file(event):
        filepath = event.data.strip("{}")
        file_label.config(text=filepath)
        file_label.filepath = filepath

    file_label.drop_target_register(DND_FILES)
    file_label.dnd_bind('<<Drop>>', drop_file)

    log_box = tk.Text(root, height=18, width=90, font=("Meiryo", 9))
    log_box.pack(padx=10, pady=10)

    def on_start():
        api_key = api_entry.get().strip()
        filepath = getattr(file_label, "filepath", None)
        if not api_key or not filepath:
            messagebox.showerror("エラー", "APIキーとExcelファイルを入力してください。")
            return
        threading.Thread(target=start_process, args=(api_key, filepath, log_box), daemon=True).start()

    tk.Button(root, text="▶ 開始", command=on_start, bg="#4CAF50", fg="white",
              font=("Meiryo", 11, "bold"), width=15, height=1).pack(pady=10)

    root.mainloop()


# ----------------------------------------
# 実行エントリーポイント
# ----------------------------------------
if __name__ == "__main__":
    create_gui()


C:\Users\taku5\AppData\Local\Temp\ipykernel_35460\3331166874.py:130: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '商品情報なし' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[i, "備考"] = error or ""


In [16]:
import requests
import time
import pandas as pd
import tkinter as tk
from tkinter import messagebox, filedialog
from tkinterdnd2 import DND_FILES, TkinterDnD
from typing import Optional, Tuple
import threading
import datetime
import os
import traceback

# ----------------------------------------
# 設定
# ----------------------------------------
DOMAIN_JP = 5
MAX_SECONDS_ALLOWED = 10
TOKEN_WAIT_TIME = 300       # 5分待機
AUTOSAVE_INTERVAL = 180     # 3分ごと自動保存
STOP_FLAG = False           # 強制停止フラグ

# ----------------------------------------
# Keepa API 呼び出し
# ----------------------------------------
def fetch_top_display_price(api_key: str, code: str) -> Tuple[Optional[str], Optional[int], Optional[str], int, int]:
    url = (
        f"https://api.keepa.com/product?key={api_key}"
        f"&domain={DOMAIN_JP}&code={code}"
        "&history=0&offers=20&onlyLiveOffers=0&buybox=1&stats=0"
    )
    try:
        resp = requests.get(url, timeout=20)
        data = resp.json()
    except Exception as e:
        return None, None, f"通信エラー: {e}", 0, 0

    if not data or "products" not in data:
        return None, None, "データなし", 0, 0

    tokens_consumed = data.get("tokensConsumed", 0)
    tokens_left = data.get("tokensLeft", 0)
    products = data["products"]

    if not products:
        return None, None, "商品情報なし", tokens_consumed, tokens_left

    product = products[0]
    title = product.get("title", "")
    stats = product.get("stats") or {}

    # --- BuyBox価格を優先 ---
    for key in ("buyBoxPrice", "buyBoxShippingPrice", "current_BUY_BOX_SHIPPING"):
        v = stats.get(key)
        if isinstance(v, (int, float)) and v > 0:
            return title, int(v), None, tokens_consumed, tokens_left

    # --- Prime優先ロジック ---
    offers = product.get("offers") or []
    order = product.get("liveOffersOrder") or []
    ordered = [offers[i] for i in order if isinstance(i, int) and i < len(offers)]
    if not ordered and offers:
        ordered = offers

    prime_offer = next((o for o in ordered if o.get("isPrime")), None)
    chosen = prime_offer or (ordered[0] if ordered else None)

    if chosen:
        price = chosen.get("price")
        ship = chosen.get("shipping") or 0
        if price and price > 0:
            total = int(price) + int(ship)
            return title, total, None, tokens_consumed, tokens_left

    return title, None, "価格情報なし", tokens_consumed, tokens_left


# ----------------------------------------
# メイン処理
# ----------------------------------------
def start_process(api_key, filepath, log_box, start_button):
    global STOP_FLAG
    STOP_FLAG = False
    start_button.config(state="disabled")

    try:
        df = pd.read_excel(filepath, header=None)
    except Exception as e:
        messagebox.showerror("読込エラー", f"Excelファイルを開けませんでした。\n{e}")
        start_button.config(state="normal")
        return

    cols = ["JAN", "商品名", "価格", "備考"]
    while len(df.columns) < 4:
        df[len(df.columns)] = ""
    df.columns = cols[:len(df.columns)]

    total = len(df)
    start_time = time.time()
    save_timer = start_time
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    output_file = f"結果_{timestamp}.xlsx"

    log_box.insert(tk.END, f"📘 ファイル読込完了: {filepath}\n🔢 全{total}件の処理を開始します。\n\n")
    log_box.update()

    try:
        for i, row in df.iterrows():
            if STOP_FLAG:
                log_box.insert(tk.END, "🛑 強制停止を検出しました。途中までを保存します...\n")
                df.to_excel(output_file, index=False)
                log_box.insert(tk.END, f"💾 途中まで保存しました → {output_file}\n")
                break

            jan = str(row["JAN"]).strip()
            old_price = row.get("価格", None)
            if pd.notna(old_price) and str(old_price).strip() != "":
                log_box.insert(tk.END, f"✅ {i+1}/{total} {jan} → 価格済み（スキップ）\n")
                log_box.update()
                continue

            title, price, error, tokens_used, tokens_left = fetch_top_display_price(api_key, jan)

            if tokens_left <= 0:
                msg = f"⚠️ トークン枯渇！{TOKEN_WAIT_TIME//60}分待機します...\n"
                log_box.insert(tk.END, msg)
                log_box.update()
                df.at[i, "備考"] = "トークン待機中（5分）"
                df.to_excel(output_file, index=False)
                time.sleep(TOKEN_WAIT_TIME)
                log_box.insert(tk.END, "⏳ 再開しました。\n")

            df.at[i, "商品名"] = title or row.get("商品名", "")
            df.at[i, "価格"] = price if price else row.get("価格", "")
            df.at[i, "備考"] = error or ""

            now = time.time()
            if now - save_timer >= AUTOSAVE_INTERVAL:
                df.to_excel(output_file, index=False)
                log_box.insert(tk.END, "💾 自動保存しました（3分経過）\n")
                log_box.update()
                save_timer = now

            remaining = total - (i + 1)
            log_box.insert(tk.END, f"🕐 {i+1}/{total} 完了 → 残り{remaining}件 | トークン残: {tokens_left}\n")
            log_box.update()
            time.sleep(1.0)

        df.to_excel(output_file, index=False)
        log_box.insert(tk.END, f"\n🎉 完了！結果を「{output_file}」に保存しました。\n")
        messagebox.showinfo("完了", f"処理が完了しました！\n結果ファイル: {output_file}")

    except Exception as e:
        df.to_excel(output_file, index=False)
        log_box.insert(tk.END, f"⚠️ エラー発生: {e}\n💾 途中まで保存しました → {output_file}\n")
        log_box.insert(tk.END, traceback.format_exc())
        messagebox.showerror("エラー", f"処理中に問題が発生しました。\n途中まで保存しました。\n{output_file}")

    finally:
        start_button.config(state="normal")


# ----------------------------------------
# GUI構築
# ----------------------------------------
def create_gui():
    root = TkinterDnD.Tk()
    root.title("Keepa価格取得ツール（フラワーさん専用・安全保存版）")
    root.geometry("750x550")

    # ×ボタンを押した時の確認
    def on_close():
        if messagebox.askyesno("確認", "本当に終了しますか？\n途中のデータは保存されます。"):
            root.destroy()

    root.protocol("WM_DELETE_WINDOW", on_close)

    tk.Label(root, text="Keepa APIキー：", font=("Meiryo", 11, "bold")).pack(anchor="w", padx=10, pady=5)
    api_entry = tk.Entry(root, width=80, show="*")
    api_entry.pack(padx=10)

    # ファイル選択 or ドロップ
    tk.Label(root, text="Excelファイルを選択またはドロップ：", font=("Meiryo", 11, "bold")).pack(anchor="w", padx=10, pady=5)
    file_label = tk.Label(root, text="（ここにドラッグ＆ドロップ または 下のボタンで選択）", bg="#f8f8f8",
                          width=85, height=2, relief="groove", font=("Meiryo", 10))
    file_label.pack(padx=10, pady=5)

    def drop_file(event):
        filepath = event.data.strip("{}")
        file_label.config(text=filepath)
        file_label.filepath = filepath

    def select_file():
        filepath = filedialog.askopenfilename(filetypes=[("Excel files", "*.xlsx *.xls")])
        if filepath:
            file_label.config(text=filepath)
            file_label.filepath = filepath

    file_label.drop_target_register(DND_FILES)
    file_label.dnd_bind('<<Drop>>', drop_file)

    tk.Button(root, text="ファイルを選択", command=select_file, font=("Meiryo", 10), width=20).pack(pady=5)

    # ログ出力エリア
    log_box = tk.Text(root, height=18, width=95, font=("Meiryo", 9))
    log_box.pack(padx=10, pady=10)

    # 強制停止
    def force_stop():
        global STOP_FLAG
        STOP_FLAG = True
        log_box.insert(tk.END, "\n🛑 強制停止ボタンが押されました。安全に終了処理を行います...\n")
        log_box.update()

    start_button = tk.Button(root, text="▶ 開始", bg="#4CAF50", fg="white", font=("Meiryo", 11, "bold"), width=15)
    start_button.pack(pady=5)

    stop_button = tk.Button(root, text="🛑 強制終了", bg="#d9534f", fg="white", font=("Meiryo", 11, "bold"),
                            width=15, command=force_stop)
    stop_button.pack(pady=5)

    def on_start():
        api_key = api_entry.get().strip()
        filepath = getattr(file_label, "filepath", None)
        if not api_key or not filepath:
            messagebox.showerror("エラー", "APIキーとExcelファイルを入力してください。")
            return
        threading.Thread(target=start_process, args=(api_key, filepath, log_box, start_button), daemon=True).start()

    start_button.config(command=on_start)
    root.mainloop()


# ----------------------------------------
# 実行
# ----------------------------------------
if __name__ == "__main__":
    create_gui()


In [17]:
import requests
import time
import pandas as pd
import tkinter as tk
from tkinter import messagebox, filedialog
from tkinterdnd2 import DND_FILES, TkinterDnD
from typing import Optional, Tuple
import threading
import datetime
import os
import traceback

# ----------------------------------------
# 設定
# ----------------------------------------
DOMAIN_JP = 5
MAX_SECONDS_ALLOWED = 10
TOKEN_WAIT_TIME = 300       # 5分待機
AUTOSAVE_INTERVAL = 180     # 3分ごと自動保存
STOP_FLAG = False           # 強制停止フラグ

# ----------------------------------------
# Keepa API 呼び出し
# ----------------------------------------
def fetch_top_display_price(api_key: str, code: str) -> Tuple[Optional[str], Optional[int], Optional[str], int, int]:
    url = (
        f"https://api.keepa.com/product?key={api_key}"
        f"&domain={DOMAIN_JP}&code={code}"
        "&history=0&offers=20&onlyLiveOffers=0&buybox=1&stats=0"
    )
    try:
        resp = requests.get(url, timeout=20)
        data = resp.json()
    except Exception as e:
        return None, None, f"通信エラー: {e}", 0, 0

    if not data or "products" not in data:
        return None, None, "データなし", 0, 0

    tokens_consumed = data.get("tokensConsumed", 0)
    tokens_left = data.get("tokensLeft", 0)
    products = data["products"]

    if not products:
        return None, None, "商品情報なし", tokens_consumed, tokens_left

    product = products[0]
    title = product.get("title", "")
    stats = product.get("stats") or {}

    # --- BuyBox価格を優先 ---
    for key in ("buyBoxPrice", "buyBoxShippingPrice", "current_BUY_BOX_SHIPPING"):
        v = stats.get(key)
        if isinstance(v, (int, float)) and v > 0:
            return title, int(v), None, tokens_consumed, tokens_left

    # --- Prime優先ロジック ---
    offers = product.get("offers") or []
    order = product.get("liveOffersOrder") or []
    ordered = [offers[i] for i in order if isinstance(i, int) and i < len(offers)]
    if not ordered and offers:
        ordered = offers

    prime_offer = next((o for o in ordered if o.get("isPrime")), None)
    chosen = prime_offer or (ordered[0] if ordered else None)

    if chosen:
        price = chosen.get("price")
        ship = chosen.get("shipping") or 0
        if price and price > 0:
            total = int(price) + int(ship)
            return title, total, None, tokens_consumed, tokens_left

    return title, None, "価格情報なし", tokens_consumed, tokens_left


# ----------------------------------------
# メイン処理
# ----------------------------------------
def start_process(api_key, filepath, log_box, start_button):
    global STOP_FLAG
    STOP_FLAG = False
    start_button.config(state="disabled")

    try:
        df = pd.read_excel(filepath, header=None)
    except Exception as e:
        messagebox.showerror("読込エラー", f"Excelファイルを開けませんでした。\n{e}")
        start_button.config(state="normal")
        return

    cols = ["JAN", "商品名", "価格", "備考"]
    while len(df.columns) < 4:
        df[len(df.columns)] = ""
    df.columns = cols[:len(df.columns)]

    total = len(df)
    start_time = time.time()
    save_timer = start_time
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    output_file = f"結果_{timestamp}.xlsx"

    log_box.insert(tk.END, f"📘 ファイル読込完了: {filepath}\n🔢 全{total}件の処理を開始します。\n\n")
    log_box.update()

    try:
        for i, row in df.iterrows():
            if STOP_FLAG:
                log_box.insert(tk.END, "🛑 強制停止を検出しました。途中までを保存します...\n")
                df.to_excel(output_file, index=False)
                log_box.insert(tk.END, f"💾 途中まで保存しました → {output_file}\n")
                break

            jan = str(row["JAN"]).strip()
            old_price = row.get("価格", None)
            if pd.notna(old_price) and str(old_price).strip() != "":
                log_box.insert(tk.END, f"✅ {i+1}/{total} {jan} → 価格済み（スキップ）\n")
                log_box.update()
                continue

            title, price, error, tokens_used, tokens_left = fetch_top_display_price(api_key, jan)

            if tokens_left <= 0:
                msg = f"⚠️ トークン枯渇！{TOKEN_WAIT_TIME//60}分待機します...\n"
                log_box.insert(tk.END, msg)
                log_box.update()
                df.at[i, "備考"] = "トークン待機中（5分）"
                df.to_excel(output_file, index=False)
                time.sleep(TOKEN_WAIT_TIME)
                log_box.insert(tk.END, "⏳ 再開しました。\n")

            # --- 価格と備考の出力 ---
            df.at[i, "商品名"] = title or row.get("商品名", "")
            if price is not None:
                df.at[i, "価格"] = price
            else:
                df.at[i, "価格"] = "None"  # ←価格取得失敗時に None を記載
            df.at[i, "備考"] = error or ""

            # --- 自動保存 ---
            now = time.time()
            if now - save_timer >= AUTOSAVE_INTERVAL:
                df.to_excel(output_file, index=False)
                log_box.insert(tk.END, "💾 自動保存しました（3分経過）\n")
                log_box.update()
                save_timer = now

            remaining = total - (i + 1)
            log_box.insert(tk.END, f"🕐 {i+1}/{total} 完了 → 残り{remaining}件 | トークン残: {tokens_left}\n")
            log_box.update()
            time.sleep(1.0)

        df.to_excel(output_file, index=False)
        log_box.insert(tk.END, f"\n🎉 完了！結果を「{output_file}」に保存しました。\n")
        messagebox.showinfo("完了", f"処理が完了しました！\n結果ファイル: {output_file}")

    except Exception as e:
        df.to_excel(output_file, index=False)
        log_box.insert(tk.END, f"⚠️ エラー発生: {e}\n💾 途中まで保存しました → {output_file}\n")
        log_box.insert(tk.END, traceback.format_exc())
        messagebox.showerror("エラー", f"処理中に問題が発生しました。\n途中まで保存しました。\n{output_file}")

    finally:
        start_button.config(state="normal")


# ----------------------------------------
# GUI構築
# ----------------------------------------
def create_gui():
    root = TkinterDnD.Tk()
    root.title("Keepa価格取得ツール（フラワーさん専用・安全保存版）")
    root.geometry("750x550")

    # ×ボタンを押した時の確認
    def on_close():
        if messagebox.askyesno("確認", "本当に終了しますか？\n途中のデータは保存されます。"):
            root.destroy()

    root.protocol("WM_DELETE_WINDOW", on_close)

    tk.Label(root, text="Keepa APIキー：", font=("Meiryo", 11, "bold")).pack(anchor="w", padx=10, pady=5)
    api_entry = tk.Entry(root, width=80, show="*")
    api_entry.pack(padx=10)

    # ファイル選択 or ドロップ
    tk.Label(root, text="Excelファイルを選択またはドロップ：", font=("Meiryo", 11, "bold")).pack(anchor="w", padx=10, pady=5)
    file_label = tk.Label(root, text="（ここにドラッグ＆ドロップ または 下のボタンで選択）", bg="#f8f8f8",
                          width=85, height=2, relief="groove", font=("Meiryo", 10))
    file_label.pack(padx=10, pady=5)

    def drop_file(event):
        filepath = event.data.strip("{}")
        file_label.config(text=filepath)
        file_label.filepath = filepath

    def select_file():
        filepath = filedialog.askopenfilename(filetypes=[("Excel files", "*.xlsx *.xls")])
        if filepath:
            file_label.config(text=filepath)
            file_label.filepath = filepath

    file_label.drop_target_register(DND_FILES)
    file_label.dnd_bind('<<Drop>>', drop_file)

    tk.Button(root, text="ファイルを選択", command=select_file, font=("Meiryo", 10), width=20).pack(pady=5)

    # ログ出力エリア
    log_box = tk.Text(root, height=18, width=95, font=("Meiryo", 9))
    log_box.pack(padx=10, pady=10)

    # 強制停止
    def force_stop():
        global STOP_FLAG
        STOP_FLAG = True
        log_box.insert(tk.END, "\n🛑 強制停止ボタンが押されました。安全に終了処理を行います...\n")
        log_box.update()

    start_button = tk.Button(root, text="▶ 開始", bg="#4CAF50", fg="white", font=("Meiryo", 11, "bold"), width=15)
    start_button.pack(pady=5)

    stop_button = tk.Button(root, text="🛑 強制終了", bg="#d9534f", fg="white", font=("Meiryo", 11, "bold"),
                            width=15, command=force_stop)
    stop_button.pack(pady=5)

    def on_start():
        api_key = api_entry.get().strip()
        filepath = getattr(file_label, "filepath", None)
        if not api_key or not filepath:
            messagebox.showerror("エラー", "APIキーとExcelファイルを入力してください。")
            return
        threading.Thread(target=start_process, args=(api_key, filepath, log_box, start_button), daemon=True).start()

    start_button.config(command=on_start)
    root.mainloop()


# ----------------------------------------
# 実行
# ----------------------------------------
if __name__ == "__main__":
    create_gui()


In [ ]:
import requests
import time
import pandas as pd
import tkinter as tk
from tkinter import messagebox, filedialog
from tkinterdnd2 import DND_FILES, TkinterDnD
from typing import Optional, Tuple
import threading
import datetime
import traceback
from openpyxl import load_workbook  # ← 追加

DOMAIN_JP = 5
TOKEN_WAIT_TIME = 300
AUTOSAVE_INTERVAL = 180
STOP_FLAG = False

# ------------------------------
# Keepa API 呼び出し
# ------------------------------
def fetch_top_display_price(api_key: str, code: str) -> Tuple[Optional[str], Optional[int], Optional[str], int, int]:
    url = (
        f"https://api.keepa.com/product?key={api_key}"
        f"&domain={DOMAIN_JP}&code={code}"
        "&history=0&offers=20&onlyLiveOffers=0&buybox=1&stats=0"
    )
    try:
        resp = requests.get(url, timeout=20)
        data = resp.json()
    except Exception as e:
        return None, None, f"通信エラー: {e}", 0, 0

    if not data or "products" not in data:
        return None, None, "データなし", 0, 0

    tokens_consumed = data.get("tokensConsumed", 0)
    tokens_left = data.get("tokensLeft", 0)
    products = data["products"]

    if not products:
        return None, None, "商品情報なし", tokens_consumed, tokens_left

    product = products[0]
    title = product.get("title", "")
    stats = product.get("stats") or {}

    # BuyBox優先
    for key in ("buyBoxPrice", "buyBoxShippingPrice", "current_BUY_BOX_SHIPPING"):
        v = stats.get(key)
        if isinstance(v, (int, float)) and v > 0:
            return title, int(v), None, tokens_consumed, tokens_left

    # Prime優先
    offers = product.get("offers") or []
    order = product.get("liveOffersOrder") or []
    ordered = [offers[i] for i in order if isinstance(i, int) and i < len(offers)]
    if not ordered and offers:
        ordered = offers

    prime_offer = next((o for o in ordered if o.get("isPrime")), None)
    chosen = prime_offer or (ordered[0] if ordered else None)

    if chosen:
        price = chosen.get("price")
        ship = chosen.get("shipping") or 0
        if price and price > 0:
            total = int(price) + int(ship)
            return title, total, None, tokens_consumed, tokens_left

    return title, None, "価格情報なし", tokens_consumed, tokens_left


# ------------------------------
# メイン処理
# ------------------------------
def start_process(api_key, filepath, log_box, start_button):
    global STOP_FLAG
    STOP_FLAG = False
    start_button.config(state="disabled")

    try:
        df = pd.read_excel(filepath)
    except Exception as e:
        messagebox.showerror("読込エラー", f"Excelファイルを開けませんでした。\n{e}")
        start_button.config(state="normal")
        return

    total = len(df)
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    output_file = f"結果_{timestamp}.xlsx"

    log_box.insert(tk.END, f"📘 ファイル読込完了: {filepath}\n🔢 全{total}件の処理を開始します。\n\n")
    log_box.update()

    try:
        # 📖 既存の書式を保持したまま新ブック作成
        wb = load_workbook(filepath)
        ws = wb.active

        for i, row in df.iterrows():
            if STOP_FLAG:
                log_box.insert(tk.END, "🛑 強制停止を検出しました。途中までを保存します...\n")
                wb.save(output_file)
                log_box.insert(tk.END, f"💾 途中まで保存しました → {output_file}\n")
                break

            jan = str(row.iloc[0]).strip()
            old_price = row.iloc[2] if len(row) > 2 else None

            if pd.notna(old_price) and str(old_price).strip() != "":
                log_box.insert(tk.END, f"✅ {i+1}/{total} {jan} → 価格済み（スキップ）\n")
                log_box.update()
                continue

            title, price, error, tokens_used, tokens_left = fetch_top_display_price(api_key, jan)

            if tokens_left <= 0:
                msg = f"⚠️ トークン枯渇！{TOKEN_WAIT_TIME//60}分待機します...\n"
                log_box.insert(tk.END, msg)
                log_box.update()
                ws.cell(row=i+2, column=4).value = "トークン待機中（5分）"
                wb.save(output_file)
                time.sleep(TOKEN_WAIT_TIME)
                log_box.insert(tk.END, "⏳ 再開しました。\n")

            # ✏️ 書式を維持したまま上書き
            if len(row) > 1:
                ws.cell(row=i+2, column=2).value = title or row.iloc[1]
            if len(row) > 2:
                ws.cell(row=i+2, column=3).value = price if price is not None else "None"
            if len(row) > 3:
                ws.cell(row=i+2, column=4).value = error or ""

            wb.save(output_file)

            remaining = total - (i + 1)
            log_box.insert(tk.END, f"🕐 {i+1}/{total} 完了 → 残り{remaining}件 | トークン残: {tokens_left}\n")
            log_box.update()
            time.sleep(1.0)

        wb.save(output_file)
        log_box.insert(tk.END, f"\n🎉 完了！結果を「{output_file}」に保存しました。\n")
        messagebox.showinfo("完了", f"処理が完了しました！\n結果ファイル: {output_file}")

    except Exception as e:
        log_box.insert(tk.END, f"⚠️ エラー発生: {e}\n")
        log_box.insert(tk.END, traceback.format_exc())
        messagebox.showerror("エラー", f"処理中に問題が発生しました。\n{output_file}")

    finally:
        start_button.config(state="normal")


# ------------------------------
# GUI構築
# ------------------------------
def create_gui():
    root = TkinterDnD.Tk()
    root.title("Keepa価格取得ツール（既存書式保持版）")
    root.geometry("750x550")

    def on_close():
        if messagebox.askyesno("確認", "本当に終了しますか？"):
            root.destroy()
    root.protocol("WM_DELETE_WINDOW", on_close)

    tk.Label(root, text="Keepa APIキー：", font=("Meiryo", 11, "bold")).pack(anchor="w", padx=10, pady=5)
    api_entry = tk.Entry(root, width=80, show="*")
    api_entry.pack(padx=10)

    tk.Label(root, text="Excelファイルを選択またはドロップ：", font=("Meiryo", 11, "bold")).pack(anchor="w", padx=10, pady=5)
    file_label = tk.Label(root, text="（ここにドラッグ＆ドロップ または 下のボタンで選択）", bg="#f8f8f8",
                          width=85, height=2, relief="groove", font=("Meiryo", 10))
    file_label.pack(padx=10, pady=5)

    def drop_file(event):
        filepath = event.data.strip("{}")
        file_label.config(text=filepath)
        file_label.filepath = filepath

    def select_file():
        filepath = filedialog.askopenfilename(filetypes=[("Excel files", "*.xlsx *.xls")])
        if filepath:
            file_label.config(text=filepath)
            file_label.filepath = filepath

    file_label.drop_target_register(DND_FILES)
    file_label.dnd_bind('<<Drop>>', drop_file)
    tk.Button(root, text="ファイルを選択", command=select_file, font=("Meiryo", 10), width=20).pack(pady=5)

    log_box = tk.Text(root, height=18, width=95, font=("Meiryo", 9))
    log_box.pack(padx=10, pady=10)

    def force_stop():
        global STOP_FLAG
        STOP_FLAG = True
        log_box.insert(tk.END, "\n🛑 強制停止ボタンが押されました。\n")
        log_box.update()

    start_button = tk.Button(root, text="▶ 開始", bg="#4CAF50", fg="white", font=("Meiryo", 11, "bold"), width=15)
    start_button.pack(pady=5)

    stop_button = tk.Button(root, text="🛑 強制終了", bg="#d9534f", fg="white", font=("Meiryo", 11, "bold"),
                            width=15, command=force_stop)
    stop_button.pack(pady=5)

    def on_start():
        api_key = api_entry.get().strip()
        filepath = getattr(file_label, "filepath", None)
        if not api_key or not filepath:
            messagebox.showerror("エラー", "APIキーとExcelファイルを入力してください。")
            return
        threading.Thread(target=start_process, args=(api_key, filepath, log_box, start_button), daemon=True).start()

    start_button.config(command=on_start)
    root.mainloop()


if __name__ == "__main__":
    create_gui()


Exception in thread Thread-12 (start_process):
Traceback (most recent call last):
  File "C:\Users\taku5\anaconda3\Lib\threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "C:\Users\taku5\anaconda3\Lib\threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\taku5\AppData\Local\Temp\ipykernel_30416\382626740.py", line 133, in start_process
  File "C:\Users\taku5\anaconda3\Lib\tkinter\__init__.py", line 872, in after
    name = self._register(callit)
           ^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\taku5\anaconda3\Lib\tkinter\__init__.py", line 1584, in _register
    self.tk.createcommand(name, f)
RuntimeError: main thread is not in main loop


In [2]:
import requests
import time
import pandas as pd
import tkinter as tk
from tkinter import messagebox, filedialog
from tkinterdnd2 import DND_FILES, TkinterDnD
from typing import Optional, Tuple
import threading
import datetime
import traceback
from openpyxl import load_workbook
import ctypes
import os

# ============================================================
# 設定値
# ============================================================
DOMAIN_JP = 5
TOKEN_WAIT_TIME = 300
STOP_FLAG = False
SAVE_INTERVAL = 10  # 10件ごとに上書き保存
PAUSE_THRESHOLD = 30  # トークン30以下で一時停止
PAUSE_TIME = 60  # 一時停止時間(秒)

# ============================================================
# スリープ防止（Windowsのみ）
# ============================================================
ES_CONTINUOUS = 0x80000000
ES_SYSTEM_REQUIRED = 0x00000001

def prevent_sleep():
    try:
        ctypes.windll.kernel32.SetThreadExecutionState(ES_CONTINUOUS | ES_SYSTEM_REQUIRED)
    except Exception:
        pass

def allow_sleep():
    try:
        ctypes.windll.kernel32.SetThreadExecutionState(ES_CONTINUOUS)
    except Exception:
        pass

# ============================================================
# Keepa API呼び出し
# ============================================================
def fetch_top_display_price(api_key: str, code: str) -> Tuple[Optional[str], Optional[int], Optional[str], int, int]:
    url = (
        f"https://api.keepa.com/product?key={api_key}"
        f"&domain={DOMAIN_JP}&code={code}"
        "&history=0&offers=20&onlyLiveOffers=0&buybox=1&stats=0"
    )
    try:
        resp = requests.get(url, timeout=20)
        data = resp.json()
    except Exception as e:
        return None, None, f"通信エラー: {e}", 0, 0

    if not data or "products" not in data:
        return None, None, "データなし", 0, 0

    tokens_consumed = data.get("tokensConsumed", 0)
    tokens_left = data.get("tokensLeft", 0)
    products = data["products"]

    if not products:
        return None, None, "商品が見つからない", tokens_consumed, tokens_left

    product = products[0]
    title = product.get("title", "")
    stats = product.get("stats") or {}

    # BuyBox優先
    for key in ("buyBoxPrice", "buyBoxShippingPrice", "current_BUY_BOX_SHIPPING"):
        v = stats.get(key)
        if isinstance(v, (int, float)) and v > 0:
            return title, int(v), None, tokens_consumed, tokens_left

    # Prime優先
    offers = product.get("offers") or []
    order = product.get("liveOffersOrder") or []
    ordered = [offers[i] for i in order if isinstance(i, int) and i < len(offers)]
    if not ordered and offers:
        ordered = offers

    prime_offer = next((o for o in ordered if o.get("isPrime")), None)
    chosen = prime_offer or (ordered[0] if ordered else None)

    if chosen:
        price = chosen.get("price")
        ship = chosen.get("shipping") or 0
        if price and price > 0:
            total = int(price) + int(ship)
            return title, total, None, tokens_consumed, tokens_left

    # 値段情報なし
    hit_count = len(offers)
    if hit_count > 0:
        return title, None, f"価格取得失敗（{hit_count}件ヒット）", tokens_consumed, tokens_left
    else:
        return title, None, "商品が見つからない", tokens_consumed, tokens_left

# ============================================================
# メイン処理
# ============================================================
def start_process(api_key, filepath, log_box, start_button):
    global STOP_FLAG
    STOP_FLAG = False
    prevent_sleep()
    start_button.config(state="disabled")

    try:
        df = pd.read_excel(filepath)
    except Exception as e:
        messagebox.showerror("読込エラー", f"Excelファイルを開けませんでした。\n{e}")
        start_button.config(state="normal")
        allow_sleep()
        return

    total = len(df)
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    desktop_path = os.path.join(os.path.expanduser("~"), "Desktop")
    output_file = os.path.join(desktop_path, f"結果_{timestamp}.xlsx")

    log_box.insert(tk.END, f"📘 ファイル読込完了: {filepath}\n🔢 全{total}件の処理を開始します。\n\n")
    log_box.see(tk.END)

    try:
        wb = load_workbook(filepath)
        ws = wb.active

        for i, row in df.iterrows():
            if STOP_FLAG:
                log_box.insert(tk.END, "🛑 強制停止を検出しました。途中まで保存します...\n")
                wb.save(output_file)
                log_box.insert(tk.END, f"💾 保存完了 → {output_file}\n")
                break

            jan = str(row.iloc[0]).strip()
            old_price = row.iloc[2] if len(row) > 2 else None

            if pd.notna(old_price) and str(old_price).strip() != "":
                log_box.insert(tk.END, f"✅ {i+1}/{total} {jan} → 価格済み（スキップ）\n")
                log_box.see(tk.END)
                continue

            title, price, error, tokens_used, tokens_left = fetch_top_display_price(api_key, jan)

            # トークン30以下で一時停止
            if tokens_left <= PAUSE_THRESHOLD:
                log_box.insert(tk.END, f"⚠️ トークン残{tokens_left} → 一時停止中...\n")
                log_box.see(tk.END)
                time.sleep(PAUSE_TIME)
                log_box.insert(tk.END, "⏳ 再開しました。\n")

            # Excel書き込み
            ws.cell(row=i+2, column=2).value = title or row.iloc[1]

            if price is not None:
                ws.cell(row=i+2, column=3).value = price
                ws.cell(row=i+2, column=4).value = ""
            else:
                # 値段が取得できなかった場合は Null を記入
                ws.cell(row=i+2, column=3).value = "Null"

                # 備考欄の色付き
                if error and "価格取得失敗" in error:
                    cell = ws.cell(row=i+2, column=4, value=error)
                    cell.font = cell.font.copy(bold=True, color="FF0000")
                elif "商品が見つからない" in str(error):
                    cell = ws.cell(row=i+2, column=4, value=error)
                    cell.font = cell.font.copy(color="808080")
                else:
                    ws.cell(row=i+2, column=4).value = error or ""

            # 10件ごとに上書き保存
            if (i + 1) % SAVE_INTERVAL == 0:
                wb.save(output_file)
                log_box.insert(tk.END, f"💾 {i+1}件完了 → 自動保存しました。\n")
                log_box.see(tk.END)

            remaining = total - (i + 1)
            log_box.insert(tk.END, f"🕐 {i+1}/{total} 完了 → 残り{remaining}件 | トークン残: {tokens_left}\n")
            log_box.see(tk.END)
            time.sleep(1.0)

        wb.save(output_file)
        log_box.insert(tk.END, f"\n🎉 完了！結果を「{output_file}」に保存しました。\n")
        log_box.see(tk.END)
        messagebox.showinfo("完了", f"処理が完了しました！\n結果ファイル: {output_file}")

    except Exception as e:
        log_box.insert(tk.END, f"⚠️ エラー発生: {e}\n")
        log_box.insert(tk.END, traceback.format_exc())
        messagebox.showerror("エラー", f"処理中に問題が発生しました。\n{output_file}")

    finally:
        start_button.config(state="normal")
        allow_sleep()

# ============================================================
# GUI構築
# ============================================================
def create_gui():
    root = TkinterDnD.Tk()
    root.title("Keepa価格取得ツール（Flower版）")
    root.geometry("375x275")
    root.configure(bg="#f5f0e6")

    def on_close():
        if messagebox.askyesno("確認", "本当に終了しますか？"):
            allow_sleep()
            root.destroy()
    root.protocol("WM_DELETE_WINDOW", on_close)

    tk.Label(root, text="Keepa APIキー：", bg="#f5f0e6", font=("Meiryo", 10, "bold")).pack(anchor="w", padx=10, pady=2)
    api_entry = tk.Entry(root, width=50, show="*")
    api_entry.pack(padx=10)

    tk.Label(root, text="Excelファイル：", bg="#f5f0e6", font=("Meiryo", 10, "bold")).pack(anchor="w", padx=10, pady=2)
    file_label = tk.Label(root, text="（ここにドラッグ＆ドロップ または 選択）", bg="white",
                          width=45, height=1, relief="groove", font=("Meiryo", 9))
    file_label.pack(padx=10, pady=2)

    def drop_file(event):
        filepath = event.data.strip("{}")
        file_label.config(text=filepath)
        file_label.filepath = filepath

    def select_file():
        filepath = filedialog.askopenfilename(filetypes=[("Excel files", "*.xlsx *.xls")])
        if filepath:
            file_label.config(text=filepath)
            file_label.filepath = filepath

    file_label.drop_target_register(DND_FILES)
    file_label.dnd_bind('<<Drop>>', drop_file)
    tk.Button(root, text="ファイルを選択", command=select_file, font=("Meiryo", 9), width=18).pack(pady=3)

    log_box = tk.Text(root, height=8, width=50, font=("Meiryo", 8))
    log_box.pack(padx=10, pady=5)

    def force_stop():
        global STOP_FLAG
        STOP_FLAG = True
        log_box.insert(tk.END, "\n🛑 強制終了ボタンが押されました。\n")
        log_box.see(tk.END)

    start_button = tk.Button(root, text="▶ 開始", bg="#4CAF50", fg="white",
                             font=("Meiryo", 10, "bold"), width=12)
    start_button.pack(side="left", padx=20, pady=3)

    stop_button = tk.Button(root, text="🛑 強制終了", bg="#d9534f", fg="white",
                            font=("Meiryo", 10, "bold"), width=12, command=force_stop)
    stop_button.pack(side="right", padx=20, pady=3)

    def on_start():
        api_key = api_entry.get().strip()
        filepath = getattr(file_label, "filepath", None)
        if not api_key or not filepath:
            messagebox.showerror("エラー", "APIキーとExcelファイルを入力してください。")
            return
        threading.Thread(target=start_process, args=(api_key, filepath, log_box, start_button), daemon=True).start()

    start_button.config(command=on_start)
    root.mainloop()

# ============================================================
# メイン
# ============================================================
if __name__ == "__main__":
    create_gui()




C:\Users\taku5\AppData\Local\Temp\ipykernel_6848\1390961335.py:168: DeprecationWarning: Call to deprecated function copy (Use copy(obj) or cell.obj = cell.obj + other).
  cell.font = cell.font.copy(bold=True, color="FF0000")


In [3]:
import requests
import time
import pandas as pd
import tkinter as tk
from tkinter import messagebox, filedialog
from tkinterdnd2 import DND_FILES, TkinterDnD
from typing import Optional, Tuple
import threading
import datetime
import traceback
from openpyxl import load_workbook
import ctypes
import os

# ============================================================
# 設定値
# ============================================================
DOMAIN_JP = 5
TOKEN_WAIT_TIME = 300
STOP_FLAG = False
SAVE_INTERVAL = 10  # 10件ごとに上書き保存
PAUSE_THRESHOLD = 30  # トークン30以下で一時停止
PAUSE_TIME = 60  # 一時停止時間(秒)

# ============================================================
# スリープ防止（Windows）
# ============================================================
ES_CONTINUOUS = 0x80000000
ES_SYSTEM_REQUIRED = 0x00000001

def prevent_sleep():
    try:
        ctypes.windll.kernel32.SetThreadExecutionState(ES_CONTINUOUS | ES_SYSTEM_REQUIRED)
    except Exception:
        pass

def allow_sleep():
    try:
        ctypes.windll.kernel32.SetThreadExecutionState(ES_CONTINUOUS)
    except Exception:
        pass

# ============================================================
# Keepa API呼び出し
# ============================================================
def fetch_top_display_price(api_key: str, code: str) -> Tuple[Optional[str], Optional[int], Optional[str], int, int]:
    url = (
        f"https://api.keepa.com/product?key={api_key}"
        f"&domain={DOMAIN_JP}&code={code}"
        "&history=0&offers=20&onlyLiveOffers=0&buybox=1&stats=0"
    )
    try:
        resp = requests.get(url, timeout=20)
        data = resp.json()
    except Exception as e:
        return None, None, f"通信エラー: {e}", 0, 0

    if not data or "products" not in data:
        return None, None, "データなし", 0, 0

    tokens_consumed = data.get("tokensConsumed", 0)
    tokens_left = data.get("tokensLeft", 0)
    products = data["products"]

    if not products:
        return None, None, "商品が見つからない", tokens_consumed, tokens_left

    product = products[0]
    title = product.get("title", "")
    stats = product.get("stats") or {}

    # BuyBox優先
    for key in ("buyBoxPrice", "buyBoxShippingPrice", "current_BUY_BOX_SHIPPING"):
        v = stats.get(key)
        if isinstance(v, (int, float)) and v > 0:
            return title, int(v), None, tokens_consumed, tokens_left

    # Prime優先
    offers = product.get("offers") or []
    order = product.get("liveOffersOrder") or []
    ordered = [offers[i] for i in order if isinstance(i, int) and i < len(offers)]
    if not ordered and offers:
        ordered = offers

    prime_offer = next((o for o in ordered if o.get("isPrime")), None)
    chosen = prime_offer or (ordered[0] if ordered else None)

    if chosen:
        price = chosen.get("price")
        ship = chosen.get("shipping") or 0
        if price and price > 0:
            total = int(price) + int(ship)
            return title, total, None, tokens_consumed, tokens_left

    # 値段情報なし
    hit_count = len(offers)
    if hit_count > 0:
        return title, None, f"価格取得失敗（{hit_count}件ヒット）", tokens_consumed, tokens_left
    else:
        return title, None, "商品が見つからない", tokens_consumed, tokens_left

# ============================================================
# メイン処理
# ============================================================
def start_process(api_key, filepath, log_box, start_button):
    global STOP_FLAG
    STOP_FLAG = False
    prevent_sleep()
    start_button.config(state="disabled")

    try:
        df = pd.read_excel(filepath)
    except Exception as e:
        messagebox.showerror("読込エラー", f"Excelファイルを開けませんでした。\n{e}")
        start_button.config(state="normal")
        allow_sleep()
        return

    total = len(df)
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    desktop_path = os.path.join(os.path.expanduser("~"), "Desktop")
    output_file = os.path.join(desktop_path, f"結果_{timestamp}.xlsx")

    log_box.insert(tk.END, f"📘 ファイル読込完了: {filepath}\n🔢 全{total}件の処理を開始します。\n\n")
    log_box.see(tk.END)

    try:
        wb = load_workbook(filepath)
        ws = wb.active

        for i, row in df.iterrows():
            if STOP_FLAG:
                log_box.insert(tk.END, "🛑 強制停止を検出しました。途中まで保存します...\n")
                wb.save(output_file)
                log_box.insert(tk.END, f"💾 保存完了 → {output_file}\n")
                break

            jan = str(row.iloc[0]).strip()
            old_price = row.iloc[2] if len(row) > 2 else None

            if pd.notna(old_price) and str(old_price).strip() != "":
                log_box.insert(tk.END, f"✅ {i+1}/{total} {jan} → 価格済み（スキップ）\n")
                log_box.see(tk.END)
                continue

            title, price, error, tokens_used, tokens_left = fetch_top_display_price(api_key, jan)

            if tokens_left <= PAUSE_THRESHOLD:
                log_box.insert(tk.END, f"⚠️ トークン残{tokens_left} → 一時停止中...\n")
                log_box.see(tk.END)
                time.sleep(PAUSE_TIME)
                log_box.insert(tk.END, "⏳ 再開しました。\n")

            ws.cell(row=i+2, column=2).value = title or row.iloc[1]

            if price is not None:
                ws.cell(row=i+2, column=3).value = price
                ws.cell(row=i+2, column=4).value = ""
            else:
                ws.cell(row=i+2, column=3).value = "Null"
                if error and "価格取得失敗" in error:
                    cell = ws.cell(row=i+2, column=4, value=error)
                    cell.font = cell.font.copy(bold=True, color="FF0000")
                elif "商品が見つからない" in str(error):
                    cell = ws.cell(row=i+2, column=4, value=error)
                    cell.font = cell.font.copy(color="808080")
                else:
                    ws.cell(row=i+2, column=4).value = error or ""

            if (i + 1) % SAVE_INTERVAL == 0:
                wb.save(output_file)
                log_box.insert(tk.END, f"💾 {i+1}件完了 → 自動保存しました。\n")
                log_box.see(tk.END)

            remaining = total - (i + 1)
            log_box.insert(tk.END, f"🕐 {i+1}/{total} 完了 → 残り{remaining}件 | トークン残: {tokens_left}\n")
            log_box.see(tk.END)
            time.sleep(1.0)

        wb.save(output_file)
        log_box.insert(tk.END, f"\n🎉 完了！結果を「{output_file}」に保存しました。\n")
        log_box.see(tk.END)
        messagebox.showinfo("完了", f"処理が完了しました！\n結果ファイル: {output_file}")

    except Exception as e:
        log_box.insert(tk.END, f"⚠️ エラー発生: {e}\n")
        log_box.insert(tk.END, traceback.format_exc())
        messagebox.showerror("エラー", f"処理中に問題が発生しました。\n{output_file}")

    finally:
        start_button.config(state="normal")
        allow_sleep()

# ============================================================
# GUI構築
# ============================================================
def create_gui():
    root = TkinterDnD.Tk()
    root.title("Keepa価格取得ツール（Flower版）")
    root.geometry("420x360")  # ← 高さ少し拡張してボタンが隠れないように
    root.configure(bg="#f5f0e6")
    root.attributes("-topmost", True)  # ← 常に最前面表示

    def on_close():
        if messagebox.askyesno("確認", "本当に終了しますか？"):
            allow_sleep()
            root.destroy()
    root.protocol("WM_DELETE_WINDOW", on_close)

    # 入力エリア
    tk.Label(root, text="Keepa APIキー：", bg="#f5f0e6", font=("Meiryo", 10, "bold")).pack(anchor="w", padx=10, pady=2)
    api_entry = tk.Entry(root, width=55, show="*")
    api_entry.pack(padx=10)

    tk.Label(root, text="Excelファイル：", bg="#f5f0e6", font=("Meiryo", 10, "bold")).pack(anchor="w", padx=10, pady=2)
    file_label = tk.Label(root, text="（ここにドラッグ＆ドロップ または 選択）", bg="white",
                          width=55, height=1, relief="groove", font=("Meiryo", 9))
    file_label.pack(padx=10, pady=2)

    def drop_file(event):
        filepath = event.data.strip("{}")
        file_label.config(text=filepath)
        file_label.filepath = filepath

    def select_file():
        filepath = filedialog.askopenfilename(filetypes=[("Excel files", "*.xlsx *.xls")])
        if filepath:
            file_label.config(text=filepath)
            file_label.filepath = filepath

    file_label.drop_target_register(DND_FILES)
    file_label.dnd_bind('<<Drop>>', drop_file)
    tk.Button(root, text="ファイルを選択", command=select_file, font=("Meiryo", 9), width=18).pack(pady=3)

    # ログエリア
    frame_log = tk.Frame(root, bg="#f5f0e6")
    frame_log.pack(padx=10, pady=5, fill="both", expand=True)
    log_box = tk.Text(frame_log, height=10, width=55, font=("Meiryo", 8))
    log_box.pack(side="left", fill="both", expand=True)
    scrollbar = tk.Scrollbar(frame_log, command=log_box.yview)
    scrollbar.pack(side="right", fill="y")
    log_box.config(yscrollcommand=scrollbar.set)

    # ボタンエリア（常に下部固定）
    frame_buttons = tk.Frame(root, bg="#f5f0e6")
    frame_buttons.pack(pady=5)

    def force_stop():
        global STOP_FLAG
        STOP_FLAG = True
        log_box.insert(tk.END, "\n🛑 強制終了ボタンが押されました。\n")
        log_box.see(tk.END)

    start_button = tk.Button(frame_buttons, text="▶ 開始", bg="#4CAF50", fg="white",
                             font=("Meiryo", 10, "bold"), width=14)
    start_button.pack(side="left", padx=15)

    stop_button = tk.Button(frame_buttons, text="🛑 強制終了", bg="#d9534f", fg="white",
                            font=("Meiryo", 10, "bold"), width=14, command=force_stop)
    stop_button.pack(side="right", padx=15)

    def on_start():
        api_key = api_entry.get().strip()
        filepath = getattr(file_label, "filepath", None)
        if not api_key or not filepath:
            messagebox.showerror("エラー", "APIキーとExcelファイルを入力してください。")
            return
        threading.Thread(target=start_process, args=(api_key, filepath, log_box, start_button), daemon=True).start()

    start_button.config(command=on_start)
    root.mainloop()

# ============================================================
# メイン
# ============================================================
if __name__ == "__main__":
    create_gui()


In [6]:
import requests
import time
import pandas as pd
import tkinter as tk
from tkinter import messagebox, filedialog
from tkinterdnd2 import DND_FILES, TkinterDnD
from typing import Optional, Tuple
import threading
import datetime
import traceback
from openpyxl import load_workbook
import ctypes
import os

# ============================================================
# 設定値
# ============================================================
DOMAIN_JP = 5
TOKEN_WAIT_TIME = 300
STOP_FLAG = False
SAVE_INTERVAL = 10  # 10件ごとに上書き保存
PAUSE_THRESHOLD = 30  # トークン30以下で一時停止
PAUSE_TIME = 60  # 一時停止時間(秒)

# ============================================================
# スリープ防止（Windows）
# ============================================================
ES_CONTINUOUS = 0x80000000
ES_SYSTEM_REQUIRED = 0x00000001

def prevent_sleep():
    try:
        ctypes.windll.kernel32.SetThreadExecutionState(ES_CONTINUOUS | ES_SYSTEM_REQUIRED)
    except Exception:
        pass

def allow_sleep():
    try:
        ctypes.windll.kernel32.SetThreadExecutionState(ES_CONTINUOUS)
    except Exception:
        pass

# ============================================================
# Keepa API呼び出し
# ============================================================
def fetch_top_display_price(api_key: str, code: str) -> Tuple[Optional[str], Optional[int], Optional[str], int, int, int]:
    url = (
        f"https://api.keepa.com/product?key={api_key}"
        f"&domain={DOMAIN_JP}&code={code}"
        "&history=0&offers=20&onlyLiveOffers=0&buybox=1&stats=0"
    )
    try:
        resp = requests.get(url, timeout=20)
        data = resp.json()
    except Exception as e:
        return None, None, f"通信エラー: {e}", 0, 0, 0

    if not data or "products" not in data:
        return None, None, "データなし", 0, 0, 0

    tokens_consumed = data.get("tokensConsumed", 0)
    tokens_left = data.get("tokensLeft", 0)
    products = data["products"]

    if not products:
        return None, None, "商品が見つからない", tokens_consumed, tokens_left, 0

    product = products[0]
    title = product.get("title", "")
    stats = product.get("stats") or {}

    # BuyBox優先
    for key in ("buyBoxPrice", "buyBoxShippingPrice", "current_BUY_BOX_SHIPPING"):
        v = stats.get(key)
        if isinstance(v, (int, float)) and v > 0:
            offers = product.get("offers") or []
            return title, int(v), None, tokens_consumed, tokens_left, len(offers)

    # Prime優先
    offers = product.get("offers") or []
    order = product.get("liveOffersOrder") or []
    ordered = [offers[i] for i in order if isinstance(i, int) and i < len(offers)]
    if not ordered and offers:
        ordered = offers

    prime_offer = next((o for o in ordered if o.get("isPrime")), None)
    chosen = prime_offer or (ordered[0] if ordered else None)

    if chosen:
        price = chosen.get("price")
        ship = chosen.get("shipping") or 0
        if price and price > 0:
            total = int(price) + int(ship)
            return title, total, None, tokens_consumed, tokens_left, len(offers)

    # 値段情報なし
    hit_count = len(offers)
    if hit_count > 0:
        return title, None, f"価格取得失敗（{hit_count}件ヒット）", tokens_consumed, tokens_left, hit_count
    else:
        return title, None, "商品が見つからない", tokens_consumed, tokens_left, 0

# ============================================================
# メイン処理
# ============================================================
def start_process(api_key, filepath, log_box, start_button):
    global STOP_FLAG
    STOP_FLAG = False
    prevent_sleep()
    start_button.config(state="disabled")

    try:
        df = pd.read_excel(filepath)
    except Exception as e:
        messagebox.showerror("読込エラー", f"Excelファイルを開けませんでした。\n{e}")
        start_button.config(state="normal")
        allow_sleep()
        return

    total = len(df)
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    desktop_path = os.path.join(os.path.expanduser("~"), "Desktop")
    output_file = os.path.join(desktop_path, f"結果_{timestamp}.xlsx")

    log_box.insert(tk.END, f"📘 ファイル読込完了: {filepath}\n🔢 全{total}件の処理を開始します。\n\n")
    log_box.see(tk.END)

    try:
        wb = load_workbook(filepath)
        ws = wb.active

        for i, row in df.iterrows():
            if STOP_FLAG:
                log_box.insert(tk.END, "🛑 強制停止を検出しました。途中まで保存します...\n")
                wb.save(output_file)
                log_box.insert(tk.END, f"💾 保存完了 → {output_file}\n")
                break

            jan = str(row.iloc[0]).strip()
            old_price = row.iloc[2] if len(row) > 2 else None

            if pd.notna(old_price) and str(old_price).strip() != "":
                log_box.insert(tk.END, f"✅ {i+1}/{total} {jan} → 価格済み（スキップ）\n")
                log_box.see(tk.END)
                continue

            title, price, error, tokens_used, tokens_left, hit_count = fetch_top_display_price(api_key, jan)

            if tokens_left <= PAUSE_THRESHOLD:
                log_box.insert(tk.END, f"⚠️ トークン残{tokens_left} → 一時停止中...\n")
                log_box.see(tk.END)
                time.sleep(PAUSE_TIME)
                log_box.insert(tk.END, "⏳ 再開しました。\n")

            ws.cell(row=i+2, column=2).value = title or row.iloc[1]

            # -------------------------------------------------
            # 出力ルール
            # -------------------------------------------------
            if price is not None:
                ws.cell(row=i+2, column=3).value = price

                # 価格が取れて、複数ヒットしていた場合
                if hit_count >= 2:
                    cell = ws.cell(row=i+2, column=4, value=f"{hit_count}件ヒット")
                    cell.font = cell.font.copy(bold=True, color="FF0000")
                else:
                    ws.cell(row=i+2, column=4).value = ""
            else:
                ws.cell(row=i+2, column=3).value = "Null"

                # ヒットあり
                if error and ("価格取得失敗" in error or "件ヒット" in error):
                    cell = ws.cell(row=i+2, column=4, value=error)
                    cell.font = cell.font.copy(bold=True, color="FF0000")

                # 0件（商品なし）
                elif "商品が見つからない" in str(error):
                    cell = ws.cell(row=i+2, column=4, value=error)
                    cell.font = cell.font.copy(color="808080")

                else:
                    ws.cell(row=i+2, column=4).value = error or ""

            # -------------------------------------------------
            # 10件ごとに自動保存
            # -------------------------------------------------
            if (i + 1) % SAVE_INTERVAL == 0:
                wb.save(output_file)
                log_box.insert(tk.END, f"💾 {i+1}件完了 → 自動保存しました。\n")
                log_box.see(tk.END)

            remaining = total - (i + 1)
            log_box.insert(tk.END, f"🕐 {i+1}/{total} 完了 → 残り{remaining}件 | トークン残: {tokens_left}\n")
            log_box.see(tk.END)
            time.sleep(1.0)

        wb.save(output_file)
        log_box.insert(tk.END, f"\n🎉 完了！結果を「{output_file}」に保存しました。\n")
        log_box.see(tk.END)
        messagebox.showinfo("完了", f"処理が完了しました！\n結果ファイル: {output_file}")

    except Exception as e:
        log_box.insert(tk.END, f"⚠️ エラー発生: {e}\n")
        log_box.insert(tk.END, traceback.format_exc())
        messagebox.showerror("エラー", f"処理中に問題が発生しました。\n{output_file}")

    finally:
        start_button.config(state="normal")
        allow_sleep()

# ============================================================
# GUI構築
# ============================================================
def create_gui():
    root = TkinterDnD.Tk()
    root.title("Keepa価格取得ツール（Flower版）")
    root.geometry("420x330")
    root.configure(bg="#f5f0e6")
    root.resizable(False, False)

    def on_close():
        if messagebox.askyesno("確認", "本当に終了しますか？"):
            allow_sleep()
            root.destroy()
    root.protocol("WM_DELETE_WINDOW", on_close)

    # チェックボックス（常に前面）
    def toggle_topmost():
        root.attributes("-topmost", var_topmost.get())

    var_topmost = tk.BooleanVar(value=True)
    chk_top = tk.Checkbutton(root, text="常に前面に表示", variable=var_topmost,
                             bg="#f5f0e6", font=("Meiryo", 9),
                             command=toggle_topmost)
    chk_top.pack(anchor="e", padx=10, pady=(3, 0))
    root.attributes("-topmost", True)

    tk.Label(root, text="Keepa APIキー：", bg="#f5f0e6", font=("Meiryo", 10, "bold")).pack(anchor="w", padx=10, pady=2)
    api_entry = tk.Entry(root, width=55, show="*")
    api_entry.pack(padx=10)

    tk.Label(root, text="Excelファイル：", bg="#f5f0e6", font=("Meiryo", 10, "bold")).pack(anchor="w", padx=10, pady=2)
    file_label = tk.Label(root, text="（ここにドラッグ＆ドロップ または 選択）", bg="white",
                          width=55, height=1, relief="groove", font=("Meiryo", 9))
    file_label.pack(padx=10, pady=2)

    def drop_file(event):
        filepath = event.data.strip("{}")
        file_label.config(text=filepath)
        file_label.filepath = filepath

    def select_file():
        filepath = filedialog.askopenfilename(filetypes=[("Excel files", "*.xlsx *.xls")])
        if filepath:
            file_label.config(text=filepath)
            file_label.filepath = filepath

    file_label.drop_target_register(DND_FILES)
    file_label.dnd_bind('<<Drop>>', drop_file)
    tk.Button(root, text="ファイルを選択", command=select_file, font=("Meiryo", 9), width=18).pack(pady=3)

    # ログエリア（小型）
    frame_log = tk.Frame(root, bg="#f5f0e6")
    frame_log.pack(padx=10, pady=(5, 0), fill="both", expand=True)
    log_box = tk.Text(frame_log, height=6, width=55, font=("Meiryo", 8))
    log_box.pack(side="left", fill="both", expand=True)
    scrollbar = tk.Scrollbar(frame_log, command=log_box.yview)
    scrollbar.pack(side="right", fill="y")
    log_box.config(yscrollcommand=scrollbar.set)

    # ボタンエリア
    frame_buttons = tk.Frame(root, bg="#f5f0e6")
    frame_buttons.pack(pady=5)

    def force_stop():
        global STOP_FLAG
        STOP_FLAG = True
        log_box.insert(tk.END, "\n🛑 強制終了ボタンが押されました。\n")
        log_box.see(tk.END)

    start_button = tk.Button(frame_buttons, text="▶ 開始", bg="#4CAF50", fg="white",
                             font=("Meiryo", 10, "bold"), width=14)
    start_button.pack(side="left", padx=15)

    stop_button = tk.Button(frame_buttons, text="🛑 強制終了", bg="#d9534f", fg="white",
                            font=("Meiryo", 10, "bold"), width=14, command=force_stop)
    stop_button.pack(side="right", padx=15)

    def on_start():
        api_key = api_entry.get().strip()
        filepath = getattr(file_label, "filepath", None)
        if not api_key or not filepath:
            messagebox.showerror("エラー", "APIキーとExcelファイルを入力してください。")
            return
        threading.Thread(target=start_process, args=(api_key, filepath, log_box, start_button), daemon=True).start()

    start_button.config(command=on_start)
    root.mainloop()

# ============================================================
# メイン
# ============================================================
if __name__ == "__main__":
    create_gui()


C:\Users\taku5\AppData\Local\Temp\ipykernel_6848\292534037.py:166: DeprecationWarning: Call to deprecated function copy (Use copy(obj) or cell.obj = cell.obj + other).
  cell.font = cell.font.copy(bold=True, color="FF0000")
C:\Users\taku5\AppData\Local\Temp\ipykernel_6848\292534037.py:175: DeprecationWarning: Call to deprecated function copy (Use copy(obj) or cell.obj = cell.obj + other).
  cell.font = cell.font.copy(bold=True, color="FF0000")


In [3]:
import requests
import time
import pandas as pd
import tkinter as tk
from tkinter import messagebox, filedialog
from tkinterdnd2 import DND_FILES, TkinterDnD
from typing import Optional, Tuple
import threading
import datetime
import traceback
from openpyxl import load_workbook
import ctypes
import os

# ============================================================
# 設定値
# ============================================================
DOMAIN_JP = 5
TOKEN_WAIT_TIME = 300
STOP_FLAG = False
SAVE_INTERVAL = 10  # 10件ごとに上書き保存
PAUSE_THRESHOLD = 10  # トークン30以下で一時停止
PAUSE_TIME = 60  # 一時停止時間(秒)

# ============================================================
# スリープ防止（Windows）
# ============================================================
ES_CONTINUOUS = 0x80000000
ES_SYSTEM_REQUIRED = 0x00000001

def prevent_sleep():
    try:
        ctypes.windll.kernel32.SetThreadExecutionState(ES_CONTINUOUS | ES_SYSTEM_REQUIRED)
    except Exception:
        pass

def allow_sleep():
    try:
        ctypes.windll.kernel32.SetThreadExecutionState(ES_CONTINUOUS)
    except Exception:
        pass

# ============================================================
# Keepa API呼び出し
# ============================================================
def fetch_top_display_price(api_key: str, code: str) -> Tuple[Optional[str], Optional[int], Optional[str], int, int]:
    url = (
        f"https://api.keepa.com/product?key={api_key}"
        f"&domain={DOMAIN_JP}&code={code}"
        "&history=0&offers=20&onlyLiveOffers=0&buybox=1&stats=0"
    )
    try:
        resp = requests.get(url, timeout=20)
        data = resp.json()
    except Exception as e:
        return None, None, f"通信エラー: {e}", 0, 0

    if not data or "products" not in data:
        return None, None, "データなし", 0, 0

    tokens_consumed = data.get("tokensConsumed", 0)
    tokens_left = data.get("tokensLeft", 0)
    products = data["products"]

    if not products:
        return None, None, "商品が見つからない", tokens_consumed, tokens_left

    product = products[0]
    title = product.get("title", "")
    stats = product.get("stats") or {}

    # BuyBox優先
    for key in ("buyBoxPrice", "buyBoxShippingPrice", "current_BUY_BOX_SHIPPING"):
        v = stats.get(key)
        if isinstance(v, (int, float)) and v > 0:
            return title, int(v), None, tokens_consumed, tokens_left

    # Prime優先
    offers = product.get("offers") or []
    order = product.get("liveOffersOrder") or []
    ordered = [offers[i] for i in order if isinstance(i, int) and i < len(offers)]
    if not ordered and offers:
        ordered = offers

    prime_offer = next((o for o in ordered if o.get("isPrime")), None)
    chosen = prime_offer or (ordered[0] if ordered else None)

    if chosen:
        price = chosen.get("price")
        ship = chosen.get("shipping") or 0
        if price and price > 0:
            total = int(price) + int(ship)
            return title, total, None, tokens_consumed, tokens_left

    # 値段情報なし
    hit_count = len(offers)
    if hit_count > 0:
        return title, None, f"価格取得失敗（{hit_count}件ヒット）", tokens_consumed, tokens_left
    else:
        return title, None, "商品が見つからない", tokens_consumed, tokens_left

# ============================================================
# メイン処理
# ============================================================
def start_process(api_key, filepath, log_box, start_button):
    global STOP_FLAG
    STOP_FLAG = False
    prevent_sleep()
    start_button.config(state="disabled")

    try:
        df = pd.read_excel(filepath)
    except Exception as e:
        messagebox.showerror("読込エラー", f"Excelファイルを開けませんでした。\n{e}")
        start_button.config(state="normal")
        allow_sleep()
        return

    total = len(df)
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    desktop_path = os.path.join(os.path.expanduser("~"), "Desktop")
    output_file = os.path.join(desktop_path, f"結果_{timestamp}.xlsx")

    log_box.insert(tk.END, f"📘 ファイル読込完了: {filepath}\n🔢 全{total}件の処理を開始します。\n\n")
    log_box.see(tk.END)

    try:
        wb = load_workbook(filepath)
        ws = wb.active

        for i, row in df.iterrows():
            if STOP_FLAG:
                log_box.insert(tk.END, "🛑 強制停止を検出しました。途中まで保存します...\n")
                wb.save(output_file)
                log_box.insert(tk.END, f"💾 保存完了 → {output_file}\n")
                break

            jan = str(row.iloc[0]).strip()
            old_price = row.iloc[2] if len(row) > 2 else None

            if pd.notna(old_price) and str(old_price).strip() != "":
                log_box.insert(tk.END, f"✅ {i+1}/{total} {jan} → 価格済み（スキップ）\n")
                log_box.see(tk.END)
                continue

            title, price, error, tokens_used, tokens_left = fetch_top_display_price(api_key, jan)

            if tokens_left <= PAUSE_THRESHOLD:
                log_box.insert(tk.END, f"⚠️ トークン残{tokens_left} → 一時停止中...\n")
                log_box.see(tk.END)
                time.sleep(PAUSE_TIME)
                log_box.insert(tk.END, "⏳ 再開しました。\n")

            ws.cell(row=i+2, column=2).value = title or row.iloc[1]

            if price is not None:
                ws.cell(row=i+2, column=3).value = price
                ws.cell(row=i+2, column=4).value = ""
            else:
                ws.cell(row=i+2, column=3).value = "Null"
                if error and "価格取得失敗" in error:
                    cell = ws.cell(row=i+2, column=4, value=error)
                    cell.font = cell.font.copy(bold=True, color="FF0000")
                elif "商品が見つからない" in str(error):
                    cell = ws.cell(row=i+2, column=4, value=error)
                    cell.font = cell.font.copy(color="808080")
                else:
                    ws.cell(row=i+2, column=4).value = error or ""

            if (i + 1) % SAVE_INTERVAL == 0:
                wb.save(output_file)
                log_box.insert(tk.END, f"💾 {i+1}件完了 → 自動保存しました。\n")
                log_box.see(tk.END)

            remaining = total - (i + 1)
            log_box.insert(tk.END, f"🕐 {i+1}/{total} 完了 → 残り{remaining}件 | トークン残: {tokens_left}\n")
            log_box.see(tk.END)
            time.sleep(1.0)

        wb.save(output_file)
        log_box.insert(tk.END, f"\n🎉 完了！結果を「{output_file}」に保存しました。\n")
        log_box.see(tk.END)
        messagebox.showinfo("完了", f"処理が完了しました！\n結果ファイル: {output_file}")

    except Exception as e:
        log_box.insert(tk.END, f"⚠️ エラー発生: {e}\n")
        log_box.insert(tk.END, traceback.format_exc())
        messagebox.showerror("エラー", f"処理中に問題が発生しました。\n{output_file}")

    finally:
        start_button.config(state="normal")
        allow_sleep()

# ============================================================
# GUI構築
# ============================================================
def create_gui():
    root = TkinterDnD.Tk()
    root.title("Keepa価格取得ツール（Flower版）")
    root.geometry("420x330")  # ボタンが確実に入るサイズ
    root.configure(bg="#f5f0e6")
    root.resizable(False, False)  # サイズ変更禁止

    def on_close():
        if messagebox.askyesno("確認", "本当に終了しますか？"):
            allow_sleep()
            root.destroy()
    root.protocol("WM_DELETE_WINDOW", on_close)

    # 最前面チェックボックス
    def toggle_topmost():
        root.attributes("-topmost", var_topmost.get())

    var_topmost = tk.BooleanVar(value=True)
    chk_top = tk.Checkbutton(root, text="常に前面に表示", variable=var_topmost,
                             bg="#f5f0e6", font=("Meiryo", 9),
                             command=toggle_topmost)
    chk_top.pack(anchor="e", padx=10, pady=(3, 0))
    root.attributes("-topmost", True)

    # 入力エリア
    tk.Label(root, text="Keepa APIキー：", bg="#f5f0e6", font=("Meiryo", 10, "bold")).pack(anchor="w", padx=10, pady=2)
    api_entry = tk.Entry(root, width=55, show="*")
    api_entry.pack(padx=10)

    tk.Label(root, text="Excelファイル：", bg="#f5f0e6", font=("Meiryo", 10, "bold")).pack(anchor="w", padx=10, pady=2)
    file_label = tk.Label(root, text="（ここにドラッグ＆ドロップ または 選択）", bg="white",
                          width=55, height=1, relief="groove", font=("Meiryo", 9))
    file_label.pack(padx=10, pady=2)

    def drop_file(event):
        filepath = event.data.strip("{}")
        file_label.config(text=filepath)
        file_label.filepath = filepath

    def select_file():
        filepath = filedialog.askopenfilename(filetypes=[("Excel files", "*.xlsx *.xls")])
        if filepath:
            file_label.config(text=filepath)
            file_label.filepath = filepath

    file_label.drop_target_register(DND_FILES)
    file_label.dnd_bind('<<Drop>>', drop_file)
    tk.Button(root, text="ファイルを選択", command=select_file, font=("Meiryo", 9), width=18).pack(pady=3)

    # ログエリア（縮小）
    frame_log = tk.Frame(root, bg="#f5f0e6")
    frame_log.pack(padx=10, pady=(5, 0), fill="both", expand=True)
    log_box = tk.Text(frame_log, height=6, width=55, font=("Meiryo", 8))
    log_box.pack(side="left", fill="both", expand=True)
    scrollbar = tk.Scrollbar(frame_log, command=log_box.yview)
    scrollbar.pack(side="right", fill="y")
    log_box.config(yscrollcommand=scrollbar.set)

    # ボタンエリア
    frame_buttons = tk.Frame(root, bg="#f5f0e6")
    frame_buttons.pack(pady=5)

    def force_stop():
        global STOP_FLAG
        STOP_FLAG = True
        log_box.insert(tk.END, "\n🛑 強制終了ボタンが押されました。\n")
        log_box.see(tk.END)

    start_button = tk.Button(frame_buttons, text="▶ 開始", bg="#4CAF50", fg="white",
                             font=("Meiryo", 10, "bold"), width=14)
    start_button.pack(side="left", padx=15)

    stop_button = tk.Button(frame_buttons, text="🛑 強制終了", bg="#d9534f", fg="white",
                            font=("Meiryo", 10, "bold"), width=14, command=force_stop)
    stop_button.pack(side="right", padx=15)

    def on_start():
        api_key = api_entry.get().strip()
        filepath = getattr(file_label, "filepath", None)
        if not api_key or not filepath:
            messagebox.showerror("エラー", "APIキーとExcelファイルを入力してください。")
            return
        threading.Thread(target=start_process, args=(api_key, filepath, log_box, start_button), daemon=True).start()

    start_button.config(command=on_start)
    root.mainloop()

# ============================================================
# メイン
# ============================================================
if __name__ == "__main__":
    create_gui()


In [10]:
import requests
import time
import pandas as pd
import tkinter as tk
from tkinter import messagebox, filedialog
from tkinterdnd2 import DND_FILES, TkinterDnD
from typing import Optional, Tuple
import threading
import datetime
import traceback
from openpyxl import load_workbook
import ctypes
import os

# ============================================================
# 設定値
# ============================================================
DOMAIN_JP = 5
TOKEN_WAIT_TIME = 300
STOP_FLAG = False
SAVE_INTERVAL = 10          # 10件ごとに上書き保存
PAUSE_THRESHOLD = 10        # トークン残がしきい値以下で一時停止
PAUSE_TIME = 60             # 一時停止時間(秒)
HAS_HEADER = True           # Excelの先頭行が見出しなら True、データなら False

# ============================================================
# スリープ防止（Windows）
# ============================================================
ES_CONTINUOUS = 0x80000000
ES_SYSTEM_REQUIRED = 0x00000001

def prevent_sleep():
    try:
        ctypes.windll.kernel32.SetThreadExecutionState(ES_CONTINUOUS | ES_SYSTEM_REQUIRED)
    except Exception:
        pass

def allow_sleep():
    try:
        ctypes.windll.kernel32.SetThreadExecutionState(ES_CONTINUOUS)
    except Exception:
        pass

# ============================================================
# 価格セルの「空欄判定」ユーティリティ
# ============================================================
def is_price_filled(v) -> bool:
    """
    価格が“入っている”かをゆるめに判定する。
    - 数値: 0より大きければ“入っている”とみなす
    - 文字列: 空文字/None/NaN/null/Null 以外なら“入っている”とみなす
    """
    if v is None:
        return False
    s = str(v).strip()
    if s == "" or s.lower() in ("nan", "none", "null"):
        return False
    try:
        # 数値として解釈でき、かつ正なら“入っている”
        return float(s) > 0
    except ValueError:
        # 数値化できないが、何か文字がある → 入っている扱い
        return True

# ============================================================
# Keepa API呼び出し
# ============================================================
def fetch_top_display_price(api_key: str, code: str) -> Tuple[Optional[str], Optional[int], Optional[str], int, int]:
    url = (
        f"https://api.keepa.com/product?key={api_key}"
        f"&domain={DOMAIN_JP}&code={code}"
        "&history=0&offers=20&onlyLiveOffers=0&buybox=1&stats=0"
    )
    try:
        resp = requests.get(url, timeout=20)
        data = resp.json()
    except Exception as e:
        return None, None, f"通信エラー: {e}", 0, 0

    if not data or "products" not in data:
        return None, None, "データなし", 0, 0

    tokens_consumed = data.get("tokensConsumed", 0)
    tokens_left = data.get("tokensLeft", 0)
    products = data["products"]

    if not products:
        return None, None, "商品が見つからない", tokens_consumed, tokens_left

    product = products[0]
    title = product.get("title", "")
    stats = product.get("stats") or {}

    # BuyBox優先
    for key in ("buyBoxPrice", "buyBoxShippingPrice", "current_BUY_BOX_SHIPPING"):
        v = stats.get(key)
        if isinstance(v, (int, float)) and v > 0:
            return title, int(v), None, tokens_consumed, tokens_left

    # Prime優先（なければ最初の出品）
    offers = product.get("offers") or []
    order = product.get("liveOffersOrder") or []
    ordered = [offers[i] for i in order if isinstance(i, int) and i < len(offers)]
    if not ordered and offers:
        ordered = offers

    prime_offer = next((o for o in ordered if o.get("isPrime")), None)
    chosen = prime_offer or (ordered[0] if ordered else None)

    if chosen:
        price = chosen.get("price")
        ship = chosen.get("shipping") or 0
        if price and price > 0:
            total = int(price) + int(ship)
            return title, total, None, tokens_consumed, tokens_left

    # 値段情報なし
    hit_count = len(offers)
    if hit_count > 0:
        return title, None, f"価格取得失敗（{hit_count}件ヒット）", tokens_consumed, tokens_left
    else:
        return title, None, "商品が見つからない", tokens_consumed, tokens_left

# ============================================================
# メイン処理
# ============================================================
def start_process(api_key, filepath, log_box, start_button):
    global STOP_FLAG
    STOP_FLAG = False
    prevent_sleep()
    start_button.config(state="disabled")

    try:
        # pandasは通常、先頭行を見出しとして読み込む（header=0）
        # 見出しなしのシートを読みたい場合は header=None に切り替える
        df = pd.read_excel(filepath, header=0 if HAS_HEADER else None)
    except Exception as e:
        messagebox.showerror("読込エラー", f"Excelファイルを開けませんでした。\n{e}")
        start_button.config(state="normal")
        allow_sleep()
        return

    total = len(df)
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    desktop_path = os.path.join(os.path.expanduser("~"), "Desktop")
    output_file = os.path.join(desktop_path, f"結果_{timestamp}.xlsx")

    log_box.insert(tk.END, f"ファイル読込完了: {filepath}\n全{total}件の処理を開始します。\n\n")
    log_box.see(tk.END)

    try:
        wb = load_workbook(filepath)
        ws = wb.active

        # 1行目も含めてループ（→ 行ごとに価格セルを見てスキップ判定）
        for i, row in df.iterrows():
            # Excel上の行番号（openpyxlは1始まり）
            excel_row = i + (2 if HAS_HEADER else 1)

            if STOP_FLAG:
                log_box.insert(tk.END, "強制停止を検出しました。途中まで保存します...\n")
                wb.save(output_file)
                log_box.insert(tk.END, f"保存完了 → {output_file}\n")
                break

            # 列の並び: 0=JAN, 1=タイトル, 2=価格, 3=備考（想定）
            jan = str(row.iloc[0]).strip() if len(row) > 0 else ""
            old_price = row.iloc[2] if len(row) > 2 else None

            # 価格が既に“入っている”場合はスキップ
            if is_price_filled(old_price):
                log_box.insert(tk.END, f"✅ {i+1}/{total} {jan} → 価格済み（スキップ）\n")
                log_box.see(tk.END)
                continue

            # ここから取得
            title, price, error, tokens_used, tokens_left = fetch_top_display_price(api_key, jan)

            # トークン残チェック
            if tokens_left <= PAUSE_THRESHOLD:
                log_box.insert(tk.END, f"⚠️ トークン残{tokens_left} → 一時停止中...\n")
                log_box.see(tk.END)
                time.sleep(PAUSE_TIME)
                log_box.insert(tk.END, "⏳ 再開しました。\n")

            # タイトル列
            ws.cell(row=excel_row, column=2).value = title or (row.iloc[1] if len(row) > 1 else "")

            # 価格／備考
            if price is not None:
                ws.cell(row=excel_row, column=3).value = price
                ws.cell(row=excel_row, column=4).value = ""
            else:
                ws.cell(row=excel_row, column=3).value = "Null"
                # 備考の色付け（openpyxlのFont.copyは環境によって非推奨のためtryで保護）
                note_text = error or ""
                try:
                    from openpyxl.styles import Font
                    if "価格取得失敗" in note_text:
                        ws.cell(row=excel_row, column=4).value = note_text
                        ws.cell(row=excel_row, column=4).font = Font(bold=True, color="FF0000")
                    elif "商品が見つからない" in note_text:
                        ws.cell(row=excel_row, column=4).value = note_text
                        ws.cell(row=excel_row, column=4).font = Font(color="808080")
                    else:
                        ws.cell(row=excel_row, column=4).value = note_text
                except Exception:
                    ws.cell(row=excel_row, column=4).value = note_text

            # オートセーブ
            if (i + 1) % SAVE_INTERVAL == 0:
                wb.save(output_file)
                log_box.insert(tk.END, f"💾 {i+1}件完了 → 自動保存しました。\n")
                log_box.see(tk.END)

            remaining = total - (i + 1)
            log_box.insert(tk.END, f"🕐 {i+1}/{total} 完了 → 残り{remaining}件 | トークン残: {tokens_left}\n")
            log_box.see(tk.END)
            time.sleep(1.0)

        # 最終保存
        wb.save(output_file)
        log_box.insert(tk.END, f"\n完了！結果を「{output_file}」に保存しました。\n")
        log_box.see(tk.END)
        messagebox.showinfo("完了", f"処理が完了しました！\n結果ファイル: {output_file}")

    except Exception as e:
        log_box.insert(tk.END, f"エラー発生: {e}\n")
        log_box.insert(tk.END, traceback.format_exc())
        messagebox.showerror("エラー", f"処理中に問題が発生しました。\n{output_file}")

    finally:
        start_button.config(state="normal")
        allow_sleep()

# ============================================================
# GUI構築
# ============================================================
def create_gui():
    root = TkinterDnD.Tk()
    root.title("Keepa価格取得ツール（Flower版）")
    root.geometry("420x330")  # ボタンが確実に入るサイズ
    root.configure(bg="#f5f0e6")
    root.resizable(False, False)

    def on_close():
        if messagebox.askyesno("確認", "本当に終了しますか？"):
            allow_sleep()
            root.destroy()
    root.protocol("WM_DELETE_WINDOW", on_close)

    # 最前面チェックボックス
    def toggle_topmost():
        root.attributes("-topmost", var_topmost.get())

    var_topmost = tk.BooleanVar(value=True)
    chk_top = tk.Checkbutton(root, text="常に前面に表示", variable=var_topmost,
                             bg="#f5f0e6", font=("Meiryo", 9),
                             command=toggle_topmost)
    chk_top.pack(anchor="e", padx=10, pady=(3, 0))
    root.attributes("-topmost", True)

    # 入力エリア
    tk.Label(root, text="Keepa APIキー：", bg="#f5f0e6", font=("Meiryo", 10, "bold")).pack(anchor="w", padx=10, pady=2)
    api_entry = tk.Entry(root, width=55, show="*")
    api_entry.pack(padx=10)

    tk.Label(root, text="Excelファイル：", bg="#f5f0e6", font=("Meiryo", 10, "bold")).pack(anchor="w", padx=10, pady=2)
    file_label = tk.Label(root, text="（ここにドラッグ＆ドロップ または 選択）", bg="white",
                          width=55, height=1, relief="groove", font=("Meiryo", 9))
    file_label.pack(padx=10, pady=2)

    def drop_file(event):
        filepath = event.data.strip("{}")
        file_label.config(text=filepath)
        file_label.filepath = filepath

    def select_file():
        filepath = filedialog.askopenfilename(filetypes=[("Excel files", "*.xlsx *.xls")])
        if filepath:
            file_label.config(text=filepath)
            file_label.filepath = filepath

    file_label.drop_target_register(DND_FILES)
    file_label.dnd_bind('<<Drop>>', drop_file)
    tk.Button(root, text="ファイルを選択", command=select_file, font=("Meiryo", 9), width=18).pack(pady=3)

    # ログエリア
    frame_log = tk.Frame(root, bg="#f5f0e6")
    frame_log.pack(padx=10, pady=(5, 0), fill="both", expand=True)
    log_box = tk.Text(frame_log, height=6, width=55, font=("Meiryo", 8))
    log_box.pack(side="left", fill="both", expand=True)
    scrollbar = tk.Scrollbar(frame_log, command=log_box.yview)
    scrollbar.pack(side="right", fill="y")
    log_box.config(yscrollcommand=scrollbar.set)

    # ボタンエリア
    frame_buttons = tk.Frame(root, bg="#f5f0e6")
    frame_buttons.pack(pady=5)

    def force_stop():
        global STOP_FLAG
        STOP_FLAG = True
        log_box.insert(tk.END, "\n強制終了ボタンが押されました。\n")
        log_box.see(tk.END)

    start_button = tk.Button(frame_buttons, text="▶ 開始", bg="#4CAF50", fg="white",
                             font=("Meiryo", 10, "bold"), width=14)
    start_button.pack(side="left", padx=15)

    stop_button = tk.Button(frame_buttons, text="■ 強制終了", bg="#d9534f", fg="white",
                            font=("Meiryo", 10, "bold"), width=14, command=force_stop)
    stop_button.pack(side="right", padx=15)

    def on_start():
        api_key = api_entry.get().strip()
        filepath = getattr(file_label, "filepath", None)
        if not api_key or not filepath:
            messagebox.showerror("エラー", "APIキーとExcelファイルを入力してください。")
            return
        threading.Thread(target=start_process, args=(api_key, filepath, log_box, start_button), daemon=True).start()

    start_button.config(command=on_start)
    root.mainloop()

# ============================================================
# メイン
# ============================================================
if __name__ == "__main__":
    create_gui()


In [11]:
import requests
import time
import pandas as pd
import tkinter as tk
from tkinter import messagebox, filedialog
from tkinterdnd2 import DND_FILES, TkinterDnD
from typing import Optional, Tuple
import threading
import datetime
import traceback
from openpyxl import load_workbook
import ctypes
import os

# ============================================================
# 設定値
# ============================================================
DOMAIN_JP = 5
TOKEN_WAIT_TIME = 300
STOP_FLAG = False
SAVE_INTERVAL = 10          # 10件ごとに上書き保存
PAUSE_THRESHOLD = 10        # トークン残がしきい値以下で一時停止
PAUSE_TIME = 60             # 一時停止時間(秒)
HAS_HEADER = True           # Excelの先頭行が見出しなら True、データなら False

# ============================================================
# スリープ防止（Windows）
# ============================================================
ES_CONTINUOUS = 0x80000000
ES_SYSTEM_REQUIRED = 0x00000001

def prevent_sleep():
    try:
        ctypes.windll.kernel32.SetThreadExecutionState(ES_CONTINUOUS | ES_SYSTEM_REQUIRED)
    except Exception:
        pass

def allow_sleep():
    try:
        ctypes.windll.kernel32.SetThreadExecutionState(ES_CONTINUOUS)
    except Exception:
        pass

# ============================================================
# 価格セルの「空欄判定」ユーティリティ
# ============================================================
def is_price_filled(v) -> bool:
    """
    価格が“入っている”かをゆるめに判定する。
    - 数値: 0より大きければ“入っている”とみなす
    - 文字列: 空文字/None/NaN/null/Null 以外なら“入っている”とみなす
    """
    if v is None:
        return False
    s = str(v).strip()
    if s == "" or s.lower() in ("nan", "none", "null"):
        return False
    try:
        # 数値として解釈でき、かつ正なら“入っている”
        return float(s) > 0
    except ValueError:
        # 数値化できないが、何か文字がある → 入っている扱い
        return True

# ============================================================
# Keepa API呼び出し
# ============================================================
def fetch_top_display_price(api_key: str, code: str) -> Tuple[Optional[str], Optional[int], Optional[str], int, int]:
    url = (
        f"https://api.keepa.com/product?key={api_key}"
        f"&domain={DOMAIN_JP}&code={code}"
        "&history=0&offers=20&onlyLiveOffers=0&buybox=1&stats=0"
    )
    try:
        resp = requests.get(url, timeout=20)
        data = resp.json()
    except Exception as e:
        return None, None, f"通信エラー: {e}", 0, 0

    if not data or "products" not in data:
        return None, None, "データなし", 0, 0

    tokens_consumed = data.get("tokensConsumed", 0)
    tokens_left = data.get("tokensLeft", 0)
    products = data["products"]

    if not products:
        return None, None, "商品が見つからない", tokens_consumed, tokens_left

    product = products[0]
    title = product.get("title", "")
    stats = product.get("stats") or {}

    # BuyBox優先
    for key in ("buyBoxPrice", "buyBoxShippingPrice", "current_BUY_BOX_SHIPPING"):
        v = stats.get(key)
        if isinstance(v, (int, float)) and v > 0:
            return title, int(v), None, tokens_consumed, tokens_left

    # Prime優先（なければ最初の出品）
    offers = product.get("offers") or []
    order = product.get("liveOffersOrder") or []
    ordered = [offers[i] for i in order if isinstance(i, int) and i < len(offers)]
    if not ordered and offers:
        ordered = offers

    prime_offer = next((o for o in ordered if o.get("isPrime")), None)
    chosen = prime_offer or (ordered[0] if ordered else None)

    if chosen:
        price = chosen.get("price")
        ship = chosen.get("shipping") or 0
        if price and price > 0:
            total = int(price) + int(ship)
            return title, total, None, tokens_consumed, tokens_left

    # 値段情報なし
    hit_count = len(offers)
    if hit_count > 0:
        return title, None, f"価格取得失敗（{hit_count}件ヒット）", tokens_consumed, tokens_left
    else:
        return title, None, "商品が見つからない", tokens_consumed, tokens_left

# ============================================================
# メイン処理
# ============================================================
def start_process(api_key, filepath, log_box, start_button):
    global STOP_FLAG
    STOP_FLAG = False
    prevent_sleep()
    start_button.config(state="disabled")

    try:
        # pandasは通常、先頭行を見出しとして読み込む（header=0）
        # 見出しなしのシートを読みたい場合は header=None に切り替える
        df = pd.read_excel(filepath, header=0 if HAS_HEADER else None)
    except Exception as e:
        messagebox.showerror("読込エラー", f"Excelファイルを開けませんでした。\n{e}")
        start_button.config(state="normal")
        allow_sleep()
        return

    total = len(df)
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    desktop_path = os.path.join(os.path.expanduser("~"), "Desktop")
    output_file = os.path.join(desktop_path, f"結果_{timestamp}.xlsx")

    log_box.insert(tk.END, f"ファイル読込完了: {filepath}\n全{total}件の処理を開始します。\n\n")
    log_box.see(tk.END)

    try:
        wb = load_workbook(filepath)
        ws = wb.active

        # 1行目も含めてループ（→ 行ごとに価格セルを見てスキップ判定）
        for i, row in df.iterrows():
            # Excel上の行番号（openpyxlは1始まり）
            excel_row = i + (2 if HAS_HEADER else 1)

            if STOP_FLAG:
                log_box.insert(tk.END, "強制停止を検出しました。途中まで保存します...\n")
                wb.save(output_file)
                log_box.insert(tk.END, f"保存完了 → {output_file}\n")
                break

            # 列の並び: 0=JAN, 1=タイトル, 2=価格, 3=備考（想定）
            jan = str(row.iloc[0]).strip() if len(row) > 0 else ""
            old_price = row.iloc[2] if len(row) > 2 else None

            # 価格が既に“入っている”場合はスキップ
            if is_price_filled(old_price):
                log_box.insert(tk.END, f"✅ {i+1}/{total} {jan} → 価格済み（スキップ）\n")
                log_box.see(tk.END)
                continue

            # ここから取得
            title, price, error, tokens_used, tokens_left = fetch_top_display_price(api_key, jan)

            # トークン残チェック
            if tokens_left <= PAUSE_THRESHOLD:
                log_box.insert(tk.END, f"⚠️ トークン残{tokens_left} → 一時停止中...\n")
                log_box.see(tk.END)
                time.sleep(PAUSE_TIME)
                log_box.insert(tk.END, "⏳ 再開しました。\n")

            # タイトル列
            ws.cell(row=excel_row, column=2).value = title or (row.iloc[1] if len(row) > 1 else "")

            # 価格／備考
            if price is not None:
                ws.cell(row=excel_row, column=3).value = price
                ws.cell(row=excel_row, column=4).value = ""
            else:
                ws.cell(row=excel_row, column=3).value = "Null"
                # 備考の色付け（openpyxlのFont.copyは環境によって非推奨のためtryで保護）
                note_text = error or ""
                try:
                    from openpyxl.styles import Font
                    if "価格取得失敗" in note_text:
                        ws.cell(row=excel_row, column=4).value = note_text
                        ws.cell(row=excel_row, column=4).font = Font(bold=True, color="FF0000")
                    elif "商品が見つからない" in note_text:
                        ws.cell(row=excel_row, column=4).value = note_text
                        ws.cell(row=excel_row, column=4).font = Font(color="808080")
                    else:
                        ws.cell(row=excel_row, column=4).value = note_text
                except Exception:
                    ws.cell(row=excel_row, column=4).value = note_text

            # オートセーブ
            if (i + 1) % SAVE_INTERVAL == 0:
                wb.save(output_file)
                log_box.insert(tk.END, f"💾 {i+1}件完了 → 自動保存しました。\n")
                log_box.see(tk.END)

            remaining = total - (i + 1)
            log_box.insert(tk.END, f"🕐 {i+1}/{total} 完了 → 残り{remaining}件 | トークン残: {tokens_left}\n")
            log_box.see(tk.END)
            time.sleep(1.0)

        # 最終保存
        wb.save(output_file)
        log_box.insert(tk.END, f"\n完了！結果を「{output_file}」に保存しました。\n")
        log_box.see(tk.END)
        messagebox.showinfo("完了", f"処理が完了しました！\n結果ファイル: {output_file}")

    except Exception as e:
        log_box.insert(tk.END, f"エラー発生: {e}\n")
        log_box.insert(tk.END, traceback.format_exc())
        messagebox.showerror("エラー", f"処理中に問題が発生しました。\n{output_file}")

    finally:
        start_button.config(state="normal")
        allow_sleep()

# ============================================================
# GUI構築
# ============================================================
def create_gui():
    root = TkinterDnD.Tk()
    root.title("Keepa価格取得ツール（Flower版）")
    root.geometry("420x330")  # ボタンが確実に入るサイズ
    root.configure(bg="#f5f0e6")
    root.resizable(False, False)

    def on_close():
        if messagebox.askyesno("確認", "本当に終了しますか？"):
            allow_sleep()
            root.destroy()
    root.protocol("WM_DELETE_WINDOW", on_close)

    # 最前面チェックボックス
    def toggle_topmost():
        root.attributes("-topmost", var_topmost.get())

    var_topmost = tk.BooleanVar(value=True)
    chk_top = tk.Checkbutton(root, text="常に前面に表示", variable=var_topmost,
                             bg="#f5f0e6", font=("Meiryo", 9),
                             command=toggle_topmost)
    chk_top.pack(anchor="e", padx=10, pady=(3, 0))
    root.attributes("-topmost", True)

    # 入力エリア
    tk.Label(root, text="Keepa APIキー：", bg="#f5f0e6", font=("Meiryo", 10, "bold")).pack(anchor="w", padx=10, pady=2)
    api_entry = tk.Entry(root, width=55, show="*")
    api_entry.pack(padx=10)

    tk.Label(root, text="Excelファイル：", bg="#f5f0e6", font=("Meiryo", 10, "bold")).pack(anchor="w", padx=10, pady=2)
    file_label = tk.Label(root, text="（ここにドラッグ＆ドロップ または 選択）", bg="white",
                          width=55, height=1, relief="groove", font=("Meiryo", 9))
    file_label.pack(padx=10, pady=2)

    def drop_file(event):
        filepath = event.data.strip("{}")
        file_label.config(text=filepath)
        file_label.filepath = filepath

    def select_file():
        filepath = filedialog.askopenfilename(filetypes=[("Excel files", "*.xlsx *.xls")])
        if filepath:
            file_label.config(text=filepath)
            file_label.filepath = filepath

    file_label.drop_target_register(DND_FILES)
    file_label.dnd_bind('<<Drop>>', drop_file)
    tk.Button(root, text="ファイルを選択", command=select_file, font=("Meiryo", 9), width=18).pack(pady=3)

    # ログエリア
    frame_log = tk.Frame(root, bg="#f5f0e6")
    frame_log.pack(padx=10, pady=(5, 0), fill="both", expand=True)
    log_box = tk.Text(frame_log, height=6, width=55, font=("Meiryo", 8))
    log_box.pack(side="left", fill="both", expand=True)
    scrollbar = tk.Scrollbar(frame_log, command=log_box.yview)
    scrollbar.pack(side="right", fill="y")
    log_box.config(yscrollcommand=scrollbar.set)

    # ボタンエリア
    frame_buttons = tk.Frame(root, bg="#f5f0e6")
    frame_buttons.pack(pady=5)

    def force_stop():
        global STOP_FLAG
        STOP_FLAG = True
        log_box.insert(tk.END, "\n強制終了ボタンが押されました。\n")
        log_box.see(tk.END)

    start_button = tk.Button(frame_buttons, text="▶ 開始", bg="#4CAF50", fg="white",
                             font=("Meiryo", 10, "bold"), width=14)
    start_button.pack(side="left", padx=15)

    stop_button = tk.Button(frame_buttons, text="■ 強制終了", bg="#d9534f", fg="white",
                            font=("Meiryo", 10, "bold"), width=14, command=force_stop)
    stop_button.pack(side="right", padx=15)

    def on_start():
        api_key = api_entry.get().strip()
        filepath = getattr(file_label, "filepath", None)
        if not api_key or not filepath:
            messagebox.showerror("エラー", "APIキーとExcelファイルを入力してください。")
            return
        threading.Thread(target=start_process, args=(api_key, filepath, log_box, start_button), daemon=True).start()

    start_button.config(command=on_start)
    root.mainloop()

# ============================================================
# メイン
# ============================================================
if __name__ == "__main__":
    create_gui()


In [12]:
import requests
import time
import pandas as pd
import tkinter as tk
from tkinter import messagebox, filedialog
from tkinterdnd2 import DND_FILES, TkinterDnD
import threading
import datetime
import traceback
from openpyxl import load_workbook
import ctypes
import os

# ============================================================
# 設定値
# ============================================================
DOMAIN_JP = 5
STOP_FLAG = False
SAVE_INTERVAL = 10
PAUSE_THRESHOLD = 10
PAUSE_TIME = 60

# ============================================================
# スリープ防止（Windows）
# ============================================================
ES_CONTINUOUS = 0x80000000
ES_SYSTEM_REQUIRED = 0x00000001

def prevent_sleep():
    try:
        ctypes.windll.kernel32.SetThreadExecutionState(ES_CONTINUOUS | ES_SYSTEM_REQUIRED)
    except Exception:
        pass

def allow_sleep():
    try:
        ctypes.windll.kernel32.SetThreadExecutionState(ES_CONTINUOUS)
    except Exception:
        pass

# ============================================================
# Keepa API呼び出し
# ============================================================
def fetch_top_display_price(api_key: str, code: str):
    url = (
        f"https://api.keepa.com/product?key={api_key}"
        f"&domain={DOMAIN_JP}&code={code}"
        "&history=0&offers=20&onlyLiveOffers=0&buybox=1&stats=0"
    )
    try:
        resp = requests.get(url, timeout=20)
        data = resp.json()
    except Exception as e:
        return None, None, f"通信エラー: {e}", 0, 0

    if not data or "products" not in data:
        return None, None, "データなし", 0, 0

    tokens_consumed = data.get("tokensConsumed", 0)
    tokens_left = data.get("tokensLeft", 0)
    products = data["products"]

    if not products:
        return None, None, "商品が見つからない", tokens_consumed, tokens_left

    product = products[0]
    title = product.get("title", "")
    stats = product.get("stats") or {}

    # BuyBox優先
    for key in ("buyBoxPrice", "buyBoxShippingPrice", "current_BUY_BOX_SHIPPING"):
        v = stats.get(key)
        if isinstance(v, (int, float)) and v > 0:
            return title, int(v), None, tokens_consumed, tokens_left

    # Prime優先
    offers = product.get("offers") or []
    order = product.get("liveOffersOrder") or []
    ordered = [offers[i] for i in order if isinstance(i, int) and i < len(offers)]
    if not ordered and offers:
        ordered = offers

    prime_offer = next((o for o in ordered if o.get("isPrime")), None)
    chosen = prime_offer or (ordered[0] if ordered else None)

    if chosen:
        price = chosen.get("price")
        ship = chosen.get("shipping") or 0
        if price and price > 0:
            total = int(price) + int(ship)
            return title, total, None, tokens_consumed, tokens_left

    hit_count = len(offers)
    if hit_count > 0:
        return title, None, f"価格取得失敗（{hit_count}件ヒット）", tokens_consumed, tokens_left
    else:
        return title, None, "商品が見つからない", tokens_consumed, tokens_left

# ============================================================
# メイン処理
# ============================================================
def start_process(api_key, filepath, log_box, start_button):
    global STOP_FLAG
    STOP_FLAG = False
    prevent_sleep()
    start_button.config(state="disabled")

    try:
        # 🔹 header=None で確実に1行目からデータとして読む
        df = pd.read_excel(filepath, header=None)
    except Exception as e:
        messagebox.showerror("読込エラー", f"Excelファイルを開けませんでした。\n{e}")
        start_button.config(state="normal")
        allow_sleep()
        return

    total = len(df)
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    desktop_path = os.path.join(os.path.expanduser("~"), "Desktop")
    output_file = os.path.join(desktop_path, f"結果_{timestamp}.xlsx")

    log_box.insert(tk.END, f"📘 ファイル読込完了: {filepath}\n🔢 全{total}件の処理を開始します。\n\n")
    log_box.see(tk.END)

    try:
        wb = load_workbook(filepath)
        ws = wb.active

        for i, row in df.iterrows():
            excel_row = i + 1  # ← Excelは1始まりなのでそのまま

            if STOP_FLAG:
                log_box.insert(tk.END, "🛑 強制停止を検出しました。途中まで保存します...\n")
                wb.save(output_file)
                log_box.insert(tk.END, f"💾 保存完了 → {output_file}\n")
                break

            jan = str(row.iloc[0]).strip() if len(row) > 0 else ""
            if jan == "" or jan.lower() == "nan":
                continue

            title, price, error, tokens_used, tokens_left = fetch_top_display_price(api_key, jan)

            if tokens_left <= PAUSE_THRESHOLD:
                log_box.insert(tk.END, f"⚠️ トークン残{tokens_left} → 一時停止中...\n")
                log_box.see(tk.END)
                time.sleep(PAUSE_TIME)
                log_box.insert(tk.END, "⏳ 再開しました。\n")

            ws.cell(row=excel_row, column=2).value = title

            if price is not None:
                ws.cell(row=excel_row, column=3).value = price
                ws.cell(row=excel_row, column=4).value = ""
            else:
                ws.cell(row=excel_row, column=3).value = "Null"
                ws.cell(row=excel_row, column=4).value = error or ""

            if (i + 1) % SAVE_INTERVAL == 0:
                wb.save(output_file)
                log_box.insert(tk.END, f"💾 {i+1}件完了 → 自動保存しました。\n")
                log_box.see(tk.END)

            log_box.insert(tk.END, f"🕐 {i+1}/{total} 完了 | トークン残: {tokens_left}\n")
            log_box.see(tk.END)
            time.sleep(1.0)

        wb.save(output_file)
        log_box.insert(tk.END, f"\n🎉 完了！結果を「{output_file}」に保存しました。\n")
        log_box.see(tk.END)
        messagebox.showinfo("完了", f"処理が完了しました！\n結果ファイル: {output_file}")

    except Exception as e:
        log_box.insert(tk.END, f"⚠️ エラー発生: {e}\n")
        log_box.insert(tk.END, traceback.format_exc())
        messagebox.showerror("エラー", f"処理中に問題が発生しました。\n{output_file}")

    finally:
        start_button.config(state="normal")
        allow_sleep()

# ============================================================
# GUI構築
# ============================================================
def create_gui():
    root = TkinterDnD.Tk()
    root.title("Keepa価格取得ツール（Flower版）")
    root.geometry("420x330")
    root.configure(bg="#f5f0e6")
    root.resizable(False, False)

    def on_close():
        if messagebox.askyesno("確認", "本当に終了しますか？"):
            allow_sleep()
            root.destroy()
    root.protocol("WM_DELETE_WINDOW", on_close)

    var_topmost = tk.BooleanVar(value=True)
    chk_top = tk.Checkbutton(root, text="常に前面に表示", variable=var_topmost,
                             bg="#f5f0e6", font=("Meiryo", 9),
                             command=lambda: root.attributes("-topmost", var_topmost.get()))
    chk_top.pack(anchor="e", padx=10, pady=(3, 0))
    root.attributes("-topmost", True)

    tk.Label(root, text="Keepa APIキー：", bg="#f5f0e6", font=("Meiryo", 10, "bold")).pack(anchor="w", padx=10, pady=2)
    api_entry = tk.Entry(root, width=55, show="*")
    api_entry.pack(padx=10)

    tk.Label(root, text="Excelファイル：", bg="#f5f0e6", font=("Meiryo", 10, "bold")).pack(anchor="w", padx=10, pady=2)
    file_label = tk.Label(root, text="（ここにドラッグ＆ドロップ または 選択）", bg="white",
                          width=55, height=1, relief="groove", font=("Meiryo", 9))
    file_label.pack(padx=10, pady=2)

    def drop_file(event):
        filepath = event.data.strip("{}")
        file_label.config(text=filepath)
        file_label.filepath = filepath

    def select_file():
        filepath = filedialog.askopenfilename(filetypes=[("Excel files", "*.xlsx *.xls")])
        if filepath:
            file_label.config(text=filepath)
            file_label.filepath = filepath

    file_label.drop_target_register(DND_FILES)
    file_label.dnd_bind('<<Drop>>', drop_file)
    tk.Button(root, text="ファイルを選択", command=select_file, font=("Meiryo", 9), width=18).pack(pady=3)

    frame_log = tk.Frame(root, bg="#f5f0e6")
    frame_log.pack(padx=10, pady=(5, 0), fill="both", expand=True)
    log_box = tk.Text(frame_log, height=6, width=55, font=("Meiryo", 8))
    log_box.pack(side="left", fill="both", expand=True)
    scrollbar = tk.Scrollbar(frame_log, command=log_box.yview)
    scrollbar.pack(side="right", fill="y")
    log_box.config(yscrollcommand=scrollbar.set)

    frame_buttons = tk.Frame(root, bg="#f5f0e6")
    frame_buttons.pack(pady=5)

    def force_stop():
        global STOP_FLAG
        STOP_FLAG = True
        log_box.insert(tk.END, "\n🛑 強制終了ボタンが押されました。\n")
        log_box.see(tk.END)

    start_button = tk.Button(frame_buttons, text="▶ 開始", bg="#4CAF50", fg="white",
                             font=("Meiryo", 10, "bold"), width=14)
    start_button.pack(side="left", padx=15)

    tk.Button(frame_buttons, text="■ 強制終了", bg="#d9534f", fg="white",
              font=("Meiryo", 10, "bold"), width=14, command=force_stop).pack(side="right", padx=15)

    start_button.config(command=lambda: threading.Thread(
        target=start_process,
        args=(api_entry.get().strip(), getattr(file_label, "filepath", None), log_box, start_button),
        daemon=True
    ).start())

    root.mainloop()

# ============================================================
# メイン
# ============================================================
if __name__ == "__main__":
    create_gui()


In [15]:
import requests
import time
import pandas as pd
import tkinter as tk
from tkinter import messagebox, filedialog
from tkinterdnd2 import DND_FILES, TkinterDnD
import threading
import datetime
import traceback
from openpyxl import load_workbook
from openpyxl.styles import Font
import ctypes
import os

# ============================================================
# 設定値
# ============================================================
DOMAIN_JP = 5
STOP_FLAG = False
SAVE_INTERVAL = 10        # 10件ごとに自動保存
PAUSE_THRESHOLD = 30      # トークン残30以下で一時停止
PAUSE_TIME = 60           # 一時停止時間（秒）

# ============================================================
# スリープ防止（Windows）
# ============================================================
ES_CONTINUOUS = 0x80000000
ES_SYSTEM_REQUIRED = 0x00000001

def prevent_sleep():
    try:
        ctypes.windll.kernel32.SetThreadExecutionState(ES_CONTINUOUS | ES_SYSTEM_REQUIRED)
    except Exception:
        pass

def allow_sleep():
    try:
        ctypes.windll.kernel32.SetThreadExecutionState(ES_CONTINUOUS)
    except Exception:
        pass

# ============================================================
# Keepa API呼び出し
# ============================================================
def fetch_top_display_price(api_key: str, code: str):
    url = (
        f"https://api.keepa.com/product?key={api_key}"
        f"&domain={DOMAIN_JP}&code={code}"
        "&history=0&offers=20&onlyLiveOffers=0&buybox=1&stats=0"
    )
    try:
        resp = requests.get(url, timeout=20)
        data = resp.json()
    except Exception as e:
        return None, None, f"通信エラー: {e}", 0, 0

    if not data or "products" not in data:
        return None, None, "データなし", 0, 0

    tokens_consumed = data.get("tokensConsumed", 0)
    tokens_left = data.get("tokensLeft", 0)
    products = data["products"]

    if not products:
        return None, None, "商品が見つからない", tokens_consumed, tokens_left

    product = products[0]
    title = product.get("title", "")
    stats = product.get("stats") or {}

    # ✅ BuyBox優先
    for key in ("buyBoxPrice", "buyBoxShippingPrice", "current_BUY_BOX_SHIPPING"):
        v = stats.get(key)
        if isinstance(v, (int, float)) and v > 0:
            return title, int(v), None, tokens_consumed, tokens_left

    # ✅ Prime優先（なければ最初の出品）
    offers = product.get("offers") or []
    order = product.get("liveOffersOrder") or []
    ordered = [offers[i] for i in order if isinstance(i, int) and i < len(offers)]
    if not ordered and offers:
        ordered = offers

    prime_offer = next((o for o in ordered if o.get("isPrime")), None)
    chosen = prime_offer or (ordered[0] if ordered else None)

    if chosen:
        price = chosen.get("price")
        ship = chosen.get("shipping") or 0
        if price and price > 0:
            total = int(price) + int(ship)
            return title, total, None, tokens_consumed, tokens_left

    # ✅ 値段情報なし
    hit_count = len(offers)
    if hit_count > 0:
        return title, None, f"価格取得失敗（{hit_count}件ヒット）", tokens_consumed, tokens_left
    else:
        return title, None, "商品が見つからない", tokens_consumed, tokens_left

# ============================================================
# メイン処理
# ============================================================
def start_process(api_key, filepath, log_box, start_button):
    global STOP_FLAG
    STOP_FLAG = False
    prevent_sleep()
    start_button.config(state="disabled")

    try:
        df = pd.read_excel(filepath, header=None)
    except Exception as e:
        messagebox.showerror("読込エラー", f"Excelファイルを開けませんでした。\n{e}")
        start_button.config(state="normal")
        allow_sleep()
        return

    total = len(df)
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    desktop_path = os.path.join(os.path.expanduser("~"), "Desktop")
    output_file = os.path.join(desktop_path, f"結果_{timestamp}.xlsx")

    log_box.insert(tk.END, f"📘 ファイル読込完了: {filepath}\n🔢 全{total}件の処理を開始します。\n\n")
    log_box.see(tk.END)

    try:
        wb = load_workbook(filepath)
        ws = wb.active

        for i, row in df.iterrows():
            excel_row = i + 1  # Excel行番号（1始まり）

            if STOP_FLAG:
                log_box.insert(tk.END, "🛑 強制停止を検出しました。途中まで保存します...\n")
                wb.save(output_file)
                log_box.insert(tk.END, f"💾 保存完了 → {output_file}\n")
                break

            jan = str(row.iloc[0]).strip() if len(row) > 0 else ""
            if jan == "" or jan.lower() == "nan":
                continue

            # ✅ 価格欄（B列＝2列目）が空欄でない場合はスキップ
            cell_price = ws.cell(row=excel_row, column=2).value
            if cell_price not in (None, "", " ", "Null"):
                log_box.insert(tk.END, f"✅ {i+1}/{total} {jan} → 価格済み（スキップ）\n")
                log_box.see(tk.END)
                continue

            title, price, error, tokens_used, tokens_left = fetch_top_display_price(api_key, jan)

            # ✅ トークン残チェック
            if tokens_left <= PAUSE_THRESHOLD:
                log_box.insert(tk.END, f"⚠️ トークン残{tokens_left} → 一時停止中...\n")
                log_box.see(tk.END)
                time.sleep(PAUSE_TIME)
                log_box.insert(tk.END, "⏳ 再開しました。\n")

            # ✅ タイトルをC列（3列目）に書き込み
            ws.cell(row=excel_row, column=3).value = title

            # ✅ 価格と備考欄（B列が価格、D列が備考）
            if price is not None:
                ws.cell(row=excel_row, column=2).value = price  # ← 価格はB列
                ws.cell(row=excel_row, column=4).value = ""
            else:
                ws.cell(row=excel_row, column=2).value = "Null"
                note = error or ""
                if "価格取得失敗" in note:
                    ws.cell(row=excel_row, column=4, value=note).font = Font(bold=True, color="FF0000")
                elif "商品が見つからない" in note:
                    ws.cell(row=excel_row, column=4, value=note).font = Font(color="808080")
                else:
                    ws.cell(row=excel_row, column=4, value=note)

            # ✅ 10件ごとに自動保存
            if (i + 1) % SAVE_INTERVAL == 0:
                wb.save(output_file)
                log_box.insert(tk.END, f"💾 {i+1}件完了 → 自動保存しました。\n")
                log_box.see(tk.END)

            log_box.insert(tk.END, f"🕐 {i+1}/{total} 完了 | トークン残: {tokens_left}\n")
            log_box.see(tk.END)
            time.sleep(1.0)

        wb.save(output_file)
        log_box.insert(tk.END, f"\n🎉 完了！結果を「{output_file}」に保存しました。\n")
        log_box.see(tk.END)
        messagebox.showinfo("完了", f"処理が完了しました！\n結果ファイル: {output_file}")

    except Exception as e:
        log_box.insert(tk.END, f"⚠️ エラー発生: {e}\n")
        log_box.insert(tk.END, traceback.format_exc())
        messagebox.showerror("エラー", f"処理中に問題が発生しました。\n{output_file}")

    finally:
        start_button.config(state="normal")
        allow_sleep()

# ============================================================
# GUI構築
# ============================================================
def create_gui():
    root = TkinterDnD.Tk()
    root.title("Keepa価格取得ツール（Flower版）")
    root.geometry("420x330")
    root.configure(bg="#f5f0e6")
    root.resizable(False, False)

    def on_close():
        if messagebox.askyesno("確認", "本当に終了しますか？"):
            allow_sleep()
            root.destroy()
    root.protocol("WM_DELETE_WINDOW", on_close)

    var_topmost = tk.BooleanVar(value=True)
    chk_top = tk.Checkbutton(root, text="常に前面に表示", variable=var_topmost,
                             bg="#f5f0e6", font=("Meiryo", 9),
                             command=lambda: root.attributes("-topmost", var_topmost.get()))
    chk_top.pack(anchor="e", padx=10, pady=(3, 0))
    root.attributes("-topmost", True)

    tk.Label(root, text="Keepa APIキー：", bg="#f5f0e6", font=("Meiryo", 10, "bold")).pack(anchor="w", padx=10, pady=2)
    api_entry = tk.Entry(root, width=55, show="*")
    api_entry.pack(padx=10)

    tk.Label(root, text="Excelファイル：", bg="#f5f0e6", font=("Meiryo", 10, "bold")).pack(anchor="w", padx=10, pady=2)
    file_label = tk.Label(root, text="（ここにドラッグ＆ドロップ または 選択）", bg="white",
                          width=55, height=1, relief="groove", font=("Meiryo", 9))
    file_label.pack(padx=10, pady=2)

    def drop_file(event):
        filepath = event.data.strip("{}")
        file_label.config(text=filepath)
        file_label.filepath = filepath

    def select_file():
        filepath = filedialog.askopenfilename(filetypes=[("Excel files", "*.xlsx *.xls")])
        if filepath:
            file_label.config(text=filepath)
            file_label.filepath = filepath

    file_label.drop_target_register(DND_FILES)
    file_label.dnd_bind('<<Drop>>', drop_file)
    tk.Button(root, text="ファイルを選択", command=select_file, font=("Meiryo", 9), width=18).pack(pady=3)

    frame_log = tk.Frame(root, bg="#f5f0e6")
    frame_log.pack(padx=10, pady=(5, 0), fill="both", expand=True)
    log_box = tk.Text(frame_log, height=6, width=55, font=("Meiryo", 8))
    log_box.pack(side="left", fill="both", expand=True)
    scrollbar = tk.Scrollbar(frame_log, command=log_box.yview)
    scrollbar.pack(side="right", fill="y")
    log_box.config(yscrollcommand=scrollbar.set)

    frame_buttons = tk.Frame(root, bg="#f5f0e6")
    frame_buttons.pack(pady=5)

    def force_stop():
        global STOP_FLAG
        STOP_FLAG = True
        log_box.insert(tk.END, "\n🛑 強制終了ボタンが押されました。\n")
        log_box.see(tk.END)

    start_button = tk.Button(frame_buttons, text="▶ 開始", bg="#4CAF50", fg="white",
                             font=("Meiryo", 10, "bold"), width=14)
    start_button.pack(side="left", padx=15)

    tk.Button(frame_buttons, text="■ 強制終了", bg="#d9534f", fg="white",
              font=("Meiryo", 10, "bold"), width=14, command=force_stop).pack(side="right", padx=15)

    start_button.config(command=lambda: threading.Thread(
        target=start_process,
        args=(api_entry.get().strip(), getattr(file_label, "filepath", None), log_box, start_button),
        daemon=True
    ).start())

    root.mainloop()

# ============================================================
# 最終チェック
# ============================================================
if __name__ == "__main__":
    create_gui()


In [ ]:
5evt1mqp5d7ju3q7kmlt8s27lp0gf8n51oird2ivf6b8oj1ko5s2ltnd2n9dgo9j